In [1]:
# ✅ FIXED: CONFIG as single source of truth

#!/usr/bin/env python3
"""
COMPREHENSIVE CLIMATE RISK ANALYSIS FOR STARBUCKS COFFEE SUPPLY CHAIN
TCFD-Aligned Framework with Physical and Transition Risk Assessment
"""

# ============================================================================
# CELL 1: IMPORTS AND CONFIGURATION
# ============================================================================

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats, interpolate
from datetime import datetime
import warnings
import os
import glob
from pathlib import Path
import geopandas as gpd
from shapely.geometry import Point
import rasterio
from rasterio.mask import mask
import json
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import folium
from folium import plugins
import networkx as nx
import pickle

warnings.filterwarnings('ignore')


In [2]:

# ============================================================================
# CONFIGURATION
# ============================================================================
CONFIG = {
    'base_path': r"C:/Users/ibeha/OneDrive/Desktop/Climate",
    'scenarios': ['historical', 'ssp126', 'ssp245', 'ssp585'],
    'variables': ['tas', 'tasmax', 'tasmin', 'pr', 'hurs'],
    'time_periods': {
        'baseline': (1985, 2014),
        'near_term': (2021, 2040),
        'mid_term': (2041, 2060)
    },
    'coffee_optimal': {
        'temp_min': 15,
        'temp_max': 24,
        'temp_optimal': 19.5,
        'precip_min': 1200,
        'precip_max': 2000,
        'precip_optimal': 1600
    },
    'elevation_lapse_rate': 0.0065,  # Temperature decrease per meter elevation
    'financial_params': {
        'discount_rate': 0.08,
        'coffee_price_baseline_usd_kg': 3.5,
        'starbucks_annual_coffee_mt': 172000,
        'operating_margin': 0.16,
        'gross_margin_pct': 0.28,
        'price_elasticity': -0.8,
        'total_assets': 35e9,
        'total_revenue': 36.2e9,
        'coffee_revenue': 36.2e9 * 0.35,
        'scope_1_2_emissions': 1.5e6,
        'scope_3_emissions': 15e6
    }
}

print("="*80)
print("STARBUCKS CLIMATE RISK ANALYSIS - COMPLETE PIPELINE")
print("="*80)
print("\n📍 Using corrected climate data with elevation adjustments")
print(f"📁 Base path: {CONFIG['base_path']}")
print(f"🌡️  Elevation lapse rate: {CONFIG['elevation_lapse_rate']*1000:.1f}°C per 1000m")
print()

STARBUCKS CLIMATE RISK ANALYSIS - COMPLETE PIPELINE

📍 Using corrected climate data with elevation adjustments
📁 Base path: C:/Users/ibeha/OneDrive/Desktop/Climate
🌡️  Elevation lapse rate: 6.5°C per 1000m



In [9]:
# ============================================================================
# STEP 1: LOAD CLIMATE DATA
# ============================================================================

def load_all_climate_data(base_path, variables, scenarios):
    """Load all CMIP6 NetCDF files"""
    climate_data = {}
    all_files = glob.glob(os.path.join(base_path, "**/*.nc"), recursive=True)
    
    for var in variables:
        var_data = {}
        for scenario in scenarios:
            matches = [f for f in all_files if f"{var}_{scenario}" in f.lower()]
            if matches:
                print(f"✅ Loading {var}_{scenario}")
                var_data[scenario] = xr.open_dataset(matches[0])
            else:
                print(f"⚠️ Missing {var}_{scenario}")
        climate_data[var] = var_data
    
    return climate_data

print("\n" + "="*80)
print("STEP 1: LOADING CLIMATE DATA")
print("="*80)
climate_data = load_all_climate_data(CONFIG['base_path'], CONFIG['variables'], CONFIG['scenarios'])
print("\n" + "="*80)
print("STEP 1: LOADING CLIMATE DATA")
print("="*80)
climate_data = load_all_climate_data(CONFIG['base_path'], CONFIG['variables'], CONFIG['scenarios'])

# Add this summary:
print("\n📊 Loading Summary:")
for var in CONFIG['variables']:
    loaded = len(climate_data.get(var, {}))
    total = len(CONFIG['scenarios'])
    print(f"  {var}: {loaded}/{total} scenarios loaded")
print()



STEP 1: LOADING CLIMATE DATA
✅ Loading tas_historical
✅ Loading tas_ssp126
✅ Loading tas_ssp245
✅ Loading tas_ssp585
✅ Loading tasmax_historical
✅ Loading tasmax_ssp126
✅ Loading tasmax_ssp245
✅ Loading tasmax_ssp585
✅ Loading tasmin_historical
✅ Loading tasmin_ssp126
✅ Loading tasmin_ssp245
✅ Loading tasmin_ssp585
✅ Loading pr_historical
✅ Loading pr_ssp126
✅ Loading pr_ssp245
✅ Loading pr_ssp585
✅ Loading hurs_historical
✅ Loading hurs_ssp126
✅ Loading hurs_ssp245
✅ Loading hurs_ssp585

STEP 1: LOADING CLIMATE DATA
✅ Loading tas_historical
✅ Loading tas_ssp126
✅ Loading tas_ssp245
✅ Loading tas_ssp585
✅ Loading tasmax_historical
✅ Loading tasmax_ssp126
✅ Loading tasmax_ssp245
✅ Loading tasmax_ssp585
✅ Loading tasmin_historical
✅ Loading tasmin_ssp126
✅ Loading tasmin_ssp245
✅ Loading tasmin_ssp585
✅ Loading pr_historical
✅ Loading pr_ssp126
✅ Loading pr_ssp245
✅ Loading pr_ssp585
✅ Loading hurs_historical
✅ Loading hurs_ssp126
✅ Loading hurs_ssp245
✅ Loading hurs_ssp585

📊 Loading S

In [7]:
# ============================================================================
# STEP 2: LOAD CONFIGURATION DATA
# ============================================================================

def load_excel_data(base_path):
    """Load Excel configuration files"""
    data = {}
    data['carbon'] = pd.read_excel(os.path.join(base_path, 'Data/carbon_prices.xlsx'))
    data['regions'] = pd.read_excel(os.path.join(base_path, 'Data/coffee_regions_coordinates.xlsx'))
    return data

print("\n" + "="*80)
print("STEP 2: LOADING CONFIGURATION DATA")
print("="*80)
excel_data = load_excel_data(CONFIG['base_path'])
regions_df = excel_data['regions']
carbon_prices = excel_data['carbon']

print(f"✅ Loaded {len(regions_df)} coffee regions")
print(f"✅ Loaded carbon price scenarios")


STEP 2: LOADING CONFIGURATION DATA
✅ Loaded 29 coffee regions
✅ Loaded carbon price scenarios


In [19]:
# ============================================================================
# STEP 3: EXTRACT REGIONAL CLIMATE DATA (WARMING DELTA METHOD) - CORRECTED WITH ELEVATION-DEPENDENT TASMAX/TASMIN
# ============================================================================
def extract_regional_climate_data(climate_data, regions_df, scenarios, time_periods):
    """Extract climate data using WARMING DELTA approach - with elevation-dependent tasmax/tasmin"""
    regional_data = {}
    
    print("\nExtracting climate data for regions:")
    print("Method: Excel baseline + CMIP6 warming delta")
    print("Temperature extremes adjusted for elevation")
    print("-" * 70)
    
    for idx, region in regions_df.iterrows():
        region_name = region['region_name']
        country = region['country']
        lat, lon = region['latitude'], region['longitude']
        
        region_id = f"{country}_{region_name}"
        elevation = (region.get('elevation_m_low', 1000) + region.get('elevation_m_high', 1000)) / 2
        
        print(f"  {region_id:40s} ({lat:6.2f}°, {lon:7.2f}°) @ {elevation:4.0f}m")
        
        regional_data[region_id] = {}
        
        # Get Excel baseline
        baseline_temp = region.get('baseline_temp_c', None)
        baseline_precip = region.get('baseline_precip_mm', None)
        
        # ============================================================
        # ELEVATION-DEPENDENT TASMAX/TASMIN OFFSETS
        # ============================================================
        # Low elevation (tropical): larger daily temperature range
        # High elevation (mountains): smaller daily temperature range
        
        if elevation < 900:
            # Very low elevation: large daily range (typically 9-11°C in tropics)
            tasmax_offset = 9.0
            tasmin_offset = -4.5
        elif elevation < 1200:
            # Low elevation: moderate-large range
            tasmax_offset = 8.0
            tasmin_offset = -4.0
        elif elevation < 1500:
            # Mid elevation: moderate range
            tasmax_offset = 6.5
            tasmin_offset = -3.5
        elif elevation < 1800:
            # High elevation: smaller range
            tasmax_offset = 5.5
            tasmin_offset = -3.0
        else:
            # Very high elevation: small range
            tasmax_offset = 5.0
            tasmin_offset = -2.5
        
        baseline_data = {}
        if pd.notna(baseline_temp):
            baseline_data['tas_baseline'] = baseline_temp
            baseline_data['tasmax_baseline'] = baseline_temp + tasmax_offset
            baseline_data['tasmin_baseline'] = baseline_temp + tasmin_offset
        
        if pd.notna(baseline_precip):
            baseline_data['pr_baseline'] = baseline_precip
        
        # Extract CMIP6 HISTORICAL
        cmip6_hist = {}
        
        # ============================================================
        # HISTORICAL TEMPERATURE (mean)
        # ============================================================
        if 'historical' in climate_data.get('tas', {}):
            ds = climate_data['tas']['historical']
            var_key = 'tas' if 'tas' in ds.variables else list(ds.data_vars)[0]
            try:
                point_data = ds[var_key].sel(lat=lat, lon=lon, method='nearest')
                time_slice = point_data.sel(time=slice('1985', '2014'))
                cmip6_hist['tas'] = float(time_slice.mean().values) - 273.15
            except:
                pass
        
        # ============================================================
        # HISTORICAL TEMPERATURE MAX
        # ============================================================
        if 'historical' in climate_data.get('tasmax', {}):
            ds = climate_data['tasmax']['historical']
            var_key = 'tasmax' if 'tasmax' in ds.variables else list(ds.data_vars)[0]
            try:
                point_data = ds[var_key].sel(lat=lat, lon=lon, method='nearest')
                time_slice = point_data.sel(time=slice('1985', '2014'))
                cmip6_hist['tasmax'] = float(time_slice.mean().values) - 273.15
            except:
                pass
        
        # ============================================================
        # HISTORICAL TEMPERATURE MIN
        # ============================================================
        if 'historical' in climate_data.get('tasmin', {}):
            ds = climate_data['tasmin']['historical']
            var_key = 'tasmin' if 'tasmin' in ds.variables else list(ds.data_vars)[0]
            try:
                point_data = ds[var_key].sel(lat=lat, lon=lon, method='nearest')
                time_slice = point_data.sel(time=slice('1985', '2014'))
                cmip6_hist['tasmin'] = float(time_slice.mean().values) - 273.15
            except:
                pass
        
        # ============================================================
        # HISTORICAL PRECIPITATION
        # ============================================================
        if 'historical' in climate_data.get('pr', {}):
            ds = climate_data['pr']['historical']
            var_key = 'pr' if 'pr' in ds.variables else list(ds.data_vars)[0]
            try:
                point_data = ds[var_key].sel(lat=lat, lon=lon, method='nearest')
                time_slice = point_data.sel(time=slice('1985', '2014'))
                cmip6_hist['pr'] = float(time_slice.mean().values) * 86400 * 365
            except:
                pass
        
        # Extract FUTURE and calculate as baseline + warming
        for scenario in ['ssp126', 'ssp245', 'ssp585']:
            regional_data[region_id][scenario] = baseline_data.copy()
            
            # ============================================================
            # TEMPERATURE (mean)
            # ============================================================
            if scenario in climate_data.get('tas', {}) and 'tas' in cmip6_hist and pd.notna(baseline_temp):
                try:
                    ds = climate_data['tas'][scenario]
                    var_key = 'tas' if 'tas' in ds.variables else list(ds.data_vars)[0]
                    point_data = ds[var_key].sel(lat=lat, lon=lon, method='nearest')
                    
                    for period_name, (start_year, end_year) in time_periods.items():
                        if period_name == 'baseline':
                            continue
                        
                        time_slice = point_data.sel(time=slice(str(start_year), str(end_year)))
                        future_cmip6 = float(time_slice.mean().values) - 273.15
                        warming = future_cmip6 - cmip6_hist['tas']
                        final_temp = baseline_temp + warming
                        regional_data[region_id][scenario][f'tas_{period_name}'] = final_temp
                except:
                    pass
            
            # ============================================================
            # TEMPERATURE MAX (with elevation-dependent offset)
            # ============================================================
            if scenario in climate_data.get('tasmax', {}) and pd.notna(baseline_temp):
                try:
                    ds = climate_data['tasmax'][scenario]
                    var_key = 'tasmax' if 'tasmax' in ds.variables else list(ds.data_vars)[0]
                    point_data = ds[var_key].sel(lat=lat, lon=lon, method='nearest')
                    
                    for period_name, (start_year, end_year) in time_periods.items():
                        if period_name == 'baseline':
                            continue
                        
                        time_slice = point_data.sel(time=slice(str(start_year), str(end_year)))
                        future_tasmax = float(time_slice.mean().values) - 273.15
                        
                        # Use tasmax from historical if available, else use tas warming
                        if 'tasmax' in cmip6_hist:
                            warming = future_tasmax - cmip6_hist['tasmax']
                        else:
                            warming = future_tasmax - cmip6_hist.get('tas', 0)
                        
                        final_tasmax = baseline_data['tasmax_baseline'] + warming
                        regional_data[region_id][scenario][f'tasmax_{period_name}'] = final_tasmax
                except:
                    pass
            
            # ============================================================
            # TEMPERATURE MIN (with elevation-dependent offset)
            # ============================================================
            if scenario in climate_data.get('tasmin', {}) and pd.notna(baseline_temp):
                try:
                    ds = climate_data['tasmin'][scenario]
                    var_key = 'tasmin' if 'tasmin' in ds.variables else list(ds.data_vars)[0]
                    point_data = ds[var_key].sel(lat=lat, lon=lon, method='nearest')
                    
                    for period_name, (start_year, end_year) in time_periods.items():
                        if period_name == 'baseline':
                            continue
                        
                        time_slice = point_data.sel(time=slice(str(start_year), str(end_year)))
                        future_tasmin = float(time_slice.mean().values) - 273.15
                        
                        # Use tasmin from historical if available, else use tas warming
                        if 'tasmin' in cmip6_hist:
                            warming = future_tasmin - cmip6_hist['tasmin']
                        else:
                            warming = future_tasmin - cmip6_hist.get('tas', 0)
                        
                        final_tasmin = baseline_data['tasmin_baseline'] + warming
                        regional_data[region_id][scenario][f'tasmin_{period_name}'] = final_tasmin
                except:
                    pass
            
            # ============================================================
            # PRECIPITATION
            # ============================================================
            if scenario in climate_data.get('pr', {}) and 'pr' in cmip6_hist and pd.notna(baseline_precip):
                try:
                    ds = climate_data['pr'][scenario]
                    var_key = 'pr' if 'pr' in ds.variables else list(ds.data_vars)[0]
                    point_data = ds[var_key].sel(lat=lat, lon=lon, method='nearest')
                    
                    for period_name, (start_year, end_year) in time_periods.items():
                        if period_name == 'baseline':
                            continue
                        
                        time_slice = point_data.sel(time=slice(str(start_year), str(end_year)))
                        future_cmip6 = float(time_slice.mean().values) * 86400 * 365
                        change = future_cmip6 - cmip6_hist['pr']
                        final_precip = baseline_precip + change
                        regional_data[region_id][scenario][f'pr_{period_name}'] = final_precip
                except:
                    pass
    
    print("\n✅ Extraction complete using warming delta method with elevation-dependent extremes")
    return regional_data


print("\n" + "="*80)
print("STEP 3: EXTRACTING REGIONAL CLIMATE DATA (CORRECTED)")
print("="*80)

regional_climate = extract_regional_climate_data(
    climate_data, 
    regions_df, 
    CONFIG['scenarios'], 
    CONFIG['time_periods']
)

print(f"\n✅ Regional extraction complete for {len(regional_climate)} regions")

# VALIDATION
print("\n" + "="*80)
print("TASMAX BASELINE - ELEVATION DEPENDENT CHECK")
print("="*80)
print(f"{'Region':<40s} {'Elevation':<12s} {'Baseline T':<12s} {'Tasmax':<12s} {'Offset':<10s}")
print("-"*86)

for idx in range(min(10, len(regions_df))):
    region = regions_df.iloc[idx]
    region_id = f"{region['country']}_{region['region_name']}"
    elevation = (region.get('elevation_m_low', 1000) + region.get('elevation_m_high', 1000)) / 2
    
    if region_id in regional_climate and 'ssp245' in regional_climate[region_id]:
        data = regional_climate[region_id]['ssp245']
        baseline_temp = data.get('tas_baseline', np.nan)
        tasmax_baseline = data.get('tasmax_baseline', np.nan)
        
        if pd.notna(baseline_temp) and pd.notna(tasmax_baseline):
            offset = tasmax_baseline - baseline_temp
            print(f"{region_id:<40s} {elevation:>10.0f}m {baseline_temp:>10.1f}°C {tasmax_baseline:>10.1f}°C {offset:>8.1f}°C")

print("="*86)


STEP 3: EXTRACTING REGIONAL CLIMATE DATA (CORRECTED)

Extracting climate data for regions:
Method: Excel baseline + CMIP6 warming delta
Temperature extremes adjusted for elevation
----------------------------------------------------------------------
  Colombia_Huila                           (  2.50°,  -75.80°) @ 1700m
  Colombia_Antioquia                       (  6.20°,  -75.60°) @ 1500m
  Colombia_Tolima                          (  4.50°,  -75.20°) @ 1600m
  Colombia_Cauca                           (  2.80°,  -76.50°) @ 1800m
  Brazil_Minas Gerais Sul                  (-21.50°,  -45.50°) @ 1100m
  Brazil_São Paulo                         (-22.80°,  -47.20°) @ 1000m
  Brazil_Espírito Santo                    (-19.50°,  -40.80°) @  900m
  Guatemala_Huehuetenango                  ( 15.50°,  -91.50°) @ 1750m
  Guatemala_Antigua                        ( 14.60°,  -90.70°) @ 1600m
  Guatemala_Atitlán                        ( 14.70°,  -91.20°) @ 1700m
  Ethiopia_Yirgacheffe                

In [21]:
print("\n" + "="*80)
print("TASMAX 2050 - WITH ELEVATION-DEPENDENT BASELINES")
print("="*80)
print(f"{'Region':<40s} {'Elevation':<12s} {'Tasmax Base':<12s} {'Tasmax 2050':<12s} {'Warming':<10s}")
print("-"*86)

for idx in range(min(10, len(regions_df))):
    region = regions_df.iloc[idx]
    region_id = f"{region['country']}_{region['region_name']}"
    elevation = (region.get('elevation_m_low', 1000) + region.get('elevation_m_high', 1000)) / 2
    
    if region_id in regional_climate and 'ssp245' in regional_climate[region_id]:
        data = regional_climate[region_id]['ssp245']
        tasmax_base = data.get('tasmax_baseline', np.nan)
        tasmax_2050 = data.get('tasmax_mid_term', np.nan)
        
        if pd.notna(tasmax_base) and pd.notna(tasmax_2050):
            warming = tasmax_2050 - tasmax_base
            print(f"{region_id:<40s} {elevation:>10.0f}m {tasmax_base:>10.1f}°C {tasmax_2050:>10.1f}°C {warming:>8.1f}°C")

print("="*86)


TASMAX 2050 - WITH ELEVATION-DEPENDENT BASELINES
Region                                   Elevation    Tasmax Base  Tasmax 2050  Warming   
--------------------------------------------------------------------------------------
Colombia_Huila                                 1700m       24.5°C       25.6°C      1.1°C
Colombia_Antioquia                             1500m       25.0°C       30.0°C      5.0°C
Colombia_Tolima                                1600m       24.7°C       26.1°C      1.4°C
Colombia_Cauca                                 1800m       23.5°C       24.8°C      1.3°C
Brazil_Minas Gerais Sul                        1100m       28.5°C       29.6°C      1.1°C
Brazil_São Paulo                               1000m       29.0°C       30.6°C      1.6°C
Brazil_Espírito Santo                           900m       29.5°C       29.8°C      0.3°C
Guatemala_Huehuetenango                        1750m       23.5°C       25.4°C      1.9°C
Guatemala_Antigua                              1600m

# ============================================================================
# STEP 3: EXTRACT REGIONAL CLIMATE DATA (WARMING DELTA METHOD)
# ============================================================================
def extract_regional_climate_data(climate_data, regions_df, scenarios, time_periods):
    """Extract climate data using WARMING DELTA approach"""
    regional_data = {}
    
    print("\nExtracting climate data for regions:")
    print("Method: Excel baseline + CMIP6 warming delta")
    print("-" * 70)
    
    for idx, region in regions_df.iterrows():
        region_name = region['region_name']
        country = region['country']
        lat, lon = region['latitude'], region['longitude']
        
        region_id = f"{country}_{region_name}"
        elevation = (region.get('elevation_m_low', 1000) + region.get('elevation_m_high', 1000)) / 2
        
        print(f"  {region_id:40s} ({lat:6.2f}°, {lon:7.2f}°) @ {elevation:4.0f}m")
        
        regional_data[region_id] = {}
        
        # Get Excel baseline
        baseline_temp = region.get('baseline_temp_c', None)
        baseline_precip = region.get('baseline_precip_mm', None)
        
        baseline_data = {}
        if pd.notna(baseline_temp):
            baseline_data['tas_baseline'] = baseline_temp
            baseline_data['tasmax_baseline'] = baseline_temp + 5.5
            baseline_data['tasmin_baseline'] = baseline_temp - 5.5
        
        if pd.notna(baseline_precip):
            baseline_data['pr_baseline'] = baseline_precip
        
        # Extract CMIP6 HISTORICAL
        cmip6_hist = {}
        
        if 'historical' in climate_data.get('tas', {}):
            ds = climate_data['tas']['historical']
            var_key = 'tas' if 'tas' in ds.variables else list(ds.data_vars)[0]
            try:
                point_data = ds[var_key].sel(lat=lat, lon=lon, method='nearest')
                time_slice = point_data.sel(time=slice('1985', '2014'))
                cmip6_hist['tas'] = float(time_slice.mean().values) - 273.15
            except:
                pass
        
        if 'historical' in climate_data.get('pr', {}):
            ds = climate_data['pr']['historical']
            var_key = 'pr' if 'pr' in ds.variables else list(ds.data_vars)[0]
            try:
                point_data = ds[var_key].sel(lat=lat, lon=lon, method='nearest')
                time_slice = point_data.sel(time=slice('1985', '2014'))
                cmip6_hist['pr'] = float(time_slice.mean().values) * 86400 * 365
            except:
                pass
        
        # Extract FUTURE and calculate as baseline + warming
        for scenario in ['ssp126', 'ssp245', 'ssp585']:
            regional_data[region_id][scenario] = baseline_data.copy()
            
            # Temperature
            if scenario in climate_data.get('tas', {}) and 'tas' in cmip6_hist and pd.notna(baseline_temp):
                try:
                    ds = climate_data['tas'][scenario]
                    var_key = 'tas' if 'tas' in ds.variables else list(ds.data_vars)[0]
                    point_data = ds[var_key].sel(lat=lat, lon=lon, method='nearest')
                    
                    for period_name, (start_year, end_year) in time_periods.items():
                        if period_name == 'baseline':
                            continue
                        
                        time_slice = point_data.sel(time=slice(str(start_year), str(end_year)))
                        future_cmip6 = float(time_slice.mean().values) - 273.15
                        warming = future_cmip6 - cmip6_hist['tas']
                        final_temp = baseline_temp + warming
                        regional_data[region_id][scenario][f'tas_{period_name}'] = final_temp
                except:
                    pass
            
            # Precipitation
            if scenario in climate_data.get('pr', {}) and 'pr' in cmip6_hist and pd.notna(baseline_precip):
                try:
                    ds = climate_data['pr'][scenario]
                    var_key = 'pr' if 'pr' in ds.variables else list(ds.data_vars)[0]
                    point_data = ds[var_key].sel(lat=lat, lon=lon, method='nearest')
                    
                    for period_name, (start_year, end_year) in time_periods.items():
                        if period_name == 'baseline':
                            continue
                        
                        time_slice = point_data.sel(time=slice(str(start_year), str(end_year)))
                        future_cmip6 = float(time_slice.mean().values) * 86400 * 365
                        change = future_cmip6 - cmip6_hist['pr']
                        final_precip = baseline_precip + change
                        regional_data[region_id][scenario][f'pr_{period_name}'] = final_precip
                except:
                    pass
    
    print("\n✅ Extraction complete using warming delta method")
    return regional_data


print("\n" + "="*80)
print("STEP 3: EXTRACTING REGIONAL CLIMATE DATA")
print("="*80)

regional_climate = extract_regional_climate_data(
    climate_data, 
    regions_df, 
    CONFIG['scenarios'], 
    CONFIG['time_periods']
)

print(f"\n✅ Regional extraction complete for {len(regional_climate)} regions")

# VALIDATION
print("\n" + "="*80)
print("BASELINE & WARMING VALIDATION")
print("="*80)
print(f"{'Region':<40s} {'Baseline':<10s} {'2050':<10s} {'Warming':<10s} {'Status':<10s}")
print("-"*80)

for idx in range(min(5, len(regions_df))):
    region = regions_df.iloc[idx]
    region_id = f"{region['country']}_{region['region_name']}"
    excel_baseline = region['baseline_temp_c']
    
    if region_id in regional_climate and 'ssp245' in regional_climate[region_id]:
        data = regional_climate[region_id]['ssp245']
        extracted_baseline = data.get('tas_baseline', np.nan)
        temp_2050 = data.get('tas_mid_term', np.nan)
        
        if pd.notna(extracted_baseline) and pd.notna(temp_2050):
            warming = temp_2050 - extracted_baseline
            baseline_match = abs(extracted_baseline - excel_baseline) < 0.5
            warming_positive = warming > 0
            
            if baseline_match and warming_positive:
                status = "✓ CORRECT"
            else:
                status = "✗ ERROR"
            
            print(f"{region_id:<40s} {extracted_baseline:>8.1f}°C {temp_2050:>8.1f}°C {warming:>8.1f}°C {status:<10s}")

print("="*80)
print("Expected: All regions show '✓ CORRECT' with positive warming")
print("="*80)

After the stress calculation, add a diagnostic to verify the fix worked:

In [25]:
# ============================================================================
# STEP 4: CALCULATE CLIMATE STRESS INDICES (WITH HEAT AND FROST)
# ============================================================================

def calculate_climate_stress(regional_climate, coffee_optimal):
    """
    Calculate temperature and precipitation stress for each region
    
    Stress Index Scale (0-1):
    - 0.0-0.2: Low stress (minimal yield impact)
    - 0.2-0.4: Moderate stress (10-20% yield loss)
    - 0.4-0.6: High stress (20-40% yield loss)
    - 0.6-0.8: Severe stress (40-60% yield loss)
    - 0.8-1.0: Critical stress (>60% yield loss, crop failure likely)
    
    Based on coffee physiology literature:
    - Temperature: Optimal 18-21°C, tolerance 15-24°C
    - Extreme heat: >26°C degrades quality, >28°C reduces yield, >32°C flower abortion
    - Precipitation: Optimal 1500-1800mm, tolerance 1200-2000mm
    """
    stress_results = {}
    
    print("\nCalculating stress indices for regions:")
    for region, scenarios in regional_climate.items():
        stress_results[region] = {}
        
        for scenario, data in scenarios.items():
            # Skip metadata
            if scenario == 'metadata':
                continue
                
            for period in ['baseline', 'near_term', 'mid_term']:
                temp_key = f'tas_{period}'
                tasmax_key = f'tasmax_{period}'
                tasmin_key = f'tasmin_{period}'
                precip_key = f'pr_{period}'
                
                temp_stress = 0
                heat_stress = 0
                frost_stress = 0
                precip_stress = 0
                
                # ============================================================
                # TEMPERATURE STRESS (mean temperature)
                # ============================================================
                if temp_key in data:
                    temp = data[temp_key]
                    
                    if temp < coffee_optimal['temp_min']:
                        # Below minimum: Linear stress increase
                        # Stress = 1.0 at 10°C (5° below minimum)
                        temp_stress = min(1.0, (coffee_optimal['temp_min'] - temp) / 5.0)
                        
                    elif temp > coffee_optimal['temp_max']:
                        # Above maximum: Linear stress increase
                        # Stress = 1.0 at 30°C (6° above maximum)
                        temp_stress = min(1.0, (temp - coffee_optimal['temp_max']) / 6.0)
                        
                    else:
                        # Within tolerance range: minimal stress
                        # Stress increases as we move away from optimal
                        optimal = coffee_optimal['temp_optimal']
                        tolerance_range = (coffee_optimal['temp_max'] - coffee_optimal['temp_min']) / 2
                        temp_stress = abs(temp - optimal) / tolerance_range * 0.15
                
                # ============================================================
                # EXTREME HEAT STRESS (tasmax - CRITICAL FOR COFFEE)
                # ============================================================
                # Coffee is very sensitive to extreme heat during flowering
                # >26°C: quality degradation begins
                # >28°C: significant yield loss
                # >32°C: flower abortion (no beans form)
                if tasmax_key in data:
                    tasmax = data[tasmax_key]
                    
                    if tasmax > 32:
                        # Critical heat: flower abortion likely
                        # Stress = 1.0 at 38°C (6° above critical threshold)
                        heat_stress = min(1.0, (tasmax - 32) / 6.0)
                        
                    elif tasmax > 28:
                        # High heat: significant yield impact
                        # Stress = 0.5 at 28°C, 1.0 at 34°C
                        heat_stress = min(1.0, (tasmax - 28) / 6.0)
                        
                    elif tasmax > 26:
                        # Moderate heat: quality degradation
                        # Stress = 0.2 at 26°C, 0.5 at 28°C
                        heat_stress = min(1.0, (tasmax - 26) / 4.0 * 0.5)
                    else:
                        heat_stress = 0
                
                # ============================================================
                # FROST/CHILLING STRESS (tasmin)
                # ============================================================
                # Frost during growing season causes permanent damage
                # Chilling hours <13°C reduce flowering intensity
                if tasmin_key in data:
                    tasmin = data[tasmin_key]
                    
                    if tasmin < 0:
                        # Hard frost: permanent tree damage
                        # Stress = 1.0 at -5°C
                        frost_stress = min(1.0, (0 - tasmin) / 5.0)
                        
                    elif tasmin < 2:
                        # Frost risk: some damage possible
                        frost_stress = 0.3 + (2 - tasmin) / 2.0 * 0.3
                        
                    elif tasmin < 13:
                        # Chilling stress: reduces flowering intensity
                        # Stress increases as nights get colder
                        frost_stress = (13 - tasmin) / 10.0 * 0.15
                    else:
                        frost_stress = 0
                
                # ============================================================
                # PRECIPITATION STRESS
                # ============================================================
                if precip_key in data:
                    precip = data[precip_key]
                    
                    if precip < coffee_optimal['precip_min']:
                        # Drought stress: Linear increase
                        # Stress = 1.0 at 600mm (50% of minimum)
                        precip_stress = min(1.0, (coffee_optimal['precip_min'] - precip) / 600.0)
                        
                    elif precip > coffee_optimal['precip_max']:
                        # Excess rainfall stress: Linear increase
                        # Stress = 1.0 at 3000mm (50% above maximum)
                        precip_stress = min(1.0, (precip - coffee_optimal['precip_max']) / 1000.0)
                        
                    else:
                        # Within tolerance range: minimal stress
                        optimal = coffee_optimal['precip_optimal']
                        tolerance_range = (coffee_optimal['precip_max'] - coffee_optimal['precip_min']) / 2
                        precip_stress = abs(precip - optimal) / tolerance_range * 0.10
                
                # ============================================================
                # COMBINED STRESS
                # ============================================================
                # Temperature components: mean (40%) + heat extremes (50%) + frost (10%)
                # Precipitation: 40%
                combined_temp_stress = (
                    temp_stress * 0.4 + 
                    heat_stress * 0.5 + 
                    frost_stress * 0.1
                )
                
                total_stress = (combined_temp_stress * 0.6) + (precip_stress * 0.4)
                
                # Store results
                stress_results[region][f'{scenario}_{period}_temp_stress'] = min(combined_temp_stress, 1.0)
                stress_results[region][f'{scenario}_{period}_heat_stress'] = min(heat_stress, 1.0)
                stress_results[region][f'{scenario}_{period}_frost_stress'] = min(frost_stress, 1.0)
                stress_results[region][f'{scenario}_{period}_precip_stress'] = min(precip_stress, 1.0)
                stress_results[region][f'{scenario}_{period}_total_stress'] = min(total_stress, 1.0)
    
    return stress_results

print("\n" + "="*80)
print("STEP 4: CALCULATING CLIMATE STRESS INDICES (WITH HEAT & FROST)")
print("="*80)
stress_data = calculate_climate_stress(regional_climate, CONFIG['coffee_optimal'])
print("✅ Stress calculation complete")

# Show updated results
print("\n" + "="*80)
print("UPDATED STRESS COMPARISON")
print("="*80)
print(f"{'Region':<40s} {'Baseline':<12s} {'2050':<12s} {'Heat Stress':<12s} {'Total':<10s}")
print("-"*86)

for region in list(regional_climate.keys())[:10]:
    if region != 'nan_nan':
        baseline_stress = stress_data[region]['ssp245_baseline_total_stress']
        stress_2050 = stress_data[region]['ssp245_mid_term_total_stress']
        heat_stress = stress_data[region]['ssp245_mid_term_heat_stress']
        
        print(f"{region:<40s} {baseline_stress:>10.3f}  {stress_2050:>10.3f}  {heat_stress:>10.3f}  {stress_2050:>8.3f}")

print("="*86)


STEP 4: CALCULATING CLIMATE STRESS INDICES (WITH HEAT & FROST)

Calculating stress indices for regions:
✅ Stress calculation complete

UPDATED STRESS COMPARISON
Region                                   Baseline     2050         Heat Stress  Total     
--------------------------------------------------------------------------------------
Colombia_Huila                                0.044       0.027       0.000     0.027
Colombia_Antioquia                            0.030       0.134       0.333     0.134
Colombia_Tolima                               0.037       0.044       0.015     0.044
Colombia_Cauca                                0.048       0.063       0.000     0.063
Brazil_Minas Gerais Sul                       0.033       0.098       0.269     0.098
Brazil_São Paulo                              0.067       0.156       0.430     0.156
Brazil_Espírito Santo                         0.111       0.137       0.301     0.137
Guatemala_Huehuetenango                       0.042       

In [26]:
print("\n" + "="*80)
print("STRESS TO YIELD LOSS CONVERSION")
print("="*80)

def stress_to_yield_loss(stress):
    """Convert stress index to yield loss percentage"""
    if stress < 0.2:
        loss = stress * 0.5 * 100
    elif stress < 0.4:
        loss = (0.10 + (stress - 0.2) * 0.75) * 100
    elif stress < 0.6:
        loss = (0.25 + (stress - 0.4) * 1.25) * 100
    elif stress < 0.8:
        loss = (0.50 + (stress - 0.6) * 1.25) * 100
    else:
        loss = min(95, (0.75 + (stress - 0.8) * 1.0) * 100)
    return loss

print(f"{'Region':<40s} {'Stress 2050':<15s} {'Yield Loss':<15s}")
print("-"*70)

for region in list(regional_climate.keys())[:10]:
    if region != 'nan_nan':
        stress_2050 = stress_data[region]['ssp245_mid_term_total_stress']
        yield_loss = stress_to_yield_loss(stress_2050)
        print(f"{region:<40s} {stress_2050:>13.3f}  {yield_loss:>13.1f}%")

print("="*70)


STRESS TO YIELD LOSS CONVERSION
Region                                   Stress 2050     Yield Loss     
----------------------------------------------------------------------
Colombia_Huila                                   0.027            1.4%
Colombia_Antioquia                               0.134            6.7%
Colombia_Tolima                                  0.044            2.2%
Colombia_Cauca                                   0.063            3.2%
Brazil_Minas Gerais Sul                          0.098            4.9%
Brazil_São Paulo                                 0.156            7.8%
Brazil_Espírito Santo                            0.137            6.8%
Guatemala_Huehuetenango                          0.039            1.9%
Guatemala_Antigua                                0.045            2.3%
Guatemala_Atitlán                                0.047            2.4%


In [34]:
# Add this at the top of Step 5, before the function
import os
os.makedirs(f"{CONFIG['base_path']}/outputs/intermediate", exist_ok=True)

below is the new code i am adding for step 5-9

In [35]:
# ============================================================================
# STEP 5 (UPGRADED): CALCULATE YIELD IMPACTS — TCFD-READY & PIPELINE-SAFE
# ============================================================================

print("\nSTEP 5: Calculating coffee yield impacts (TCFD-ready)...")
print("-"*70)

def convert_stress_to_yield_loss(stress_value):
    """
    Convert stress index (0–1) to yield loss fraction (0–0.95).
    Piecewise response preserves your original assumptions.
    """
    if stress_value < 0.2:
        loss = stress_value * 0.5
    elif stress_value < 0.4:
        loss = 0.10 + (stress_value - 0.2) * 0.75
    elif stress_value < 0.6:
        loss = 0.25 + (stress_value - 0.4) * 1.25
    elif stress_value < 0.8:
        loss = 0.50 + (stress_value - 0.6) * 1.25
    else:
        loss = 0.75 + (stress_value - 0.8) * 1.0

    # Hard cap at 95% to avoid unrealistic collapse
    return min(loss, 0.95)


yield_results = []

# Helpful mappings for TCFD-style reporting
scenario_names = {
    "ssp126": "Net-Zero 2050",
    "ssp245": "Delayed Transition",
    "ssp585": "Current Policies"
}
period_to_year = {
    "near_term": 2030,
    "mid_term": 2050
}

# Loop through stress_data results (from your earlier modules)
for region, stress_values in stress_data.items():

    # Parse region id format "Country_RegionName"
    region_parts = region.split('_', 1)
    country = region_parts[0]
    region_name = region_parts[1] if len(region_parts) > 1 else region_parts[0]

    # Pull region volumes from regions_df safely
    region_info = regions_df[
        (regions_df['country'] == country) &
        (regions_df['region_name'] == region_name)
    ]

    if region_info.empty:
        continue

    row0 = region_info.iloc[0]

    # ---- PIPELINE-SAFE COLUMN FALLBACKS ----
    # Your notebook sometimes uses midpoint columns. If absent, fall back cleanly.
    if 'annual_mt_midpoint' in regions_df.columns:
        annual_mt = row0['annual_mt_midpoint']
    elif 'annual_mt' in regions_df.columns:
        annual_mt = row0['annual_mt']
    else:
        # last-ditch fallback
        annual_mt = row0.get('annual_volume_mt', np.nan)

    if 'sourcing_pct_midpoint' in regions_df.columns:
        sourcing_pct = row0['sourcing_pct_midpoint']
    elif 'sourcing_pct' in regions_df.columns:
        sourcing_pct = row0['sourcing_pct']
    else:
        sourcing_pct = row0.get('sourcing_pct', np.nan)

    # Process each scenario/period stress value
    for key, stress_value in stress_values.items():

        if 'total_stress' not in key:
            continue

        # Parse key like: "ssp245_near_term_total_stress"
        parts = key.replace('_total_stress', '').split('_')
        scenario = parts[0]
        period = '_'.join(parts[1:])

        if scenario == 'historical' or period == 'baseline':
            continue

        # Convert stress → yield loss fraction
        yield_loss_pct = convert_stress_to_yield_loss(stress_value)

        # Volume at risk
        volume_at_risk_mt = annual_mt * yield_loss_pct

        # TCFD-friendly risk buckets (based on yield loss)
        if yield_loss_pct >= 0.80:
            risk_bucket = "Extreme Risk (≥80% loss)"
        elif yield_loss_pct >= 0.60:
            risk_bucket = "Very High Risk (60–80%)"
        elif yield_loss_pct >= 0.40:
            risk_bucket = "High Risk (40–60%)"
        elif yield_loss_pct >= 0.20:
            risk_bucket = "Moderate Risk (20–40%)"
        else:
            risk_bucket = "Low Risk (<20%)"

        yield_results.append({
            # ---- DO NOT RENAME: used downstream ----
            'country': country,
            'region_name': region_name,
            'scenario': scenario,
            'period': period,
            'stress_index': stress_value,
            'yield_loss_pct': yield_loss_pct,
            'volume_at_risk_mt': volume_at_risk_mt,
            'annual_mt': annual_mt,
            'sourcing_pct': sourcing_pct,

            # ---- NEW (TCFD helpful, safe to add) ----
            'scenario_name': scenario_names.get(scenario, scenario),
            'year': period_to_year.get(period, np.nan),
            'risk_bucket': risk_bucket
        })

# Create DataFrame
yield_df = pd.DataFrame(yield_results)

print(f"✓ Calculated yield impacts for {len(yield_df)} region-scenario records.\n")

# Show most vulnerable regions (2050, SSP585)
print("Most vulnerable regions (2050, Current Policies scenario):")
vulnerable = yield_df[
    (yield_df['scenario'] == 'ssp585') &
    (yield_df['period'] == 'mid_term')
].nlargest(5, 'yield_loss_pct')

for _, row in vulnerable.iterrows():
    impact_pct = -row['yield_loss_pct'] * 100
    print(f"  • {row['region_name']}, {row['country']}: {impact_pct:.1f}% loss "
          f"({row['volume_at_risk_mt']:.0f} MT)")

print()

# Save results (same outputs as before)
output_dir = f"{CONFIG['base_path']}/outputs/intermediate"
os.makedirs(output_dir, exist_ok=True)

with open(f"{output_dir}/yield_impacts.pkl", 'wb') as f:
    pickle.dump(yield_df, f)

yield_df.to_csv(f"{CONFIG['base_path']}/outputs/yield_impacts_summary.csv", index=False)

print(f"✓ Saved yield impacts to outputs/")



STEP 5: Calculating coffee yield impacts (TCFD-ready)...
----------------------------------------------------------------------
✓ Calculated yield impacts for 168 region-scenario records.

Most vulnerable regions (2050, Current Policies scenario):
  • Sumatra Aceh, Indonesia: -23.5% loss (1625 MT)
  • Central Highlands, Vietnam: -19.5% loss (508 MT)
  • Java West, Indonesia: -14.8% loss (503 MT)
  • Yunnan Pu er, China: -9.0% loss (307 MT)
  • Antioquia, Colombia: -8.6% loss (809 MT)

✓ Saved yield impacts to outputs/


To check the code worked

In [36]:
print()
print("="*80)
print("YIELD LOSS VALIDATION")
print("="*80)
print("Checking if yield losses are reasonable after elevation correction:\n")

# Check 2050, SSP245 scenario (most likely)
check_df = yield_df[(yield_df['scenario'] == 'ssp245') & (yield_df['period'] == 'mid_term')]

print(f"SSP2-4.5, 2050 - Yield Loss Distribution:")
print(f"  Average yield loss: {check_df['yield_loss_pct'].mean()*100:.1f}%")
print(f"  Median yield loss:  {check_df['yield_loss_pct'].median()*100:.1f}%")
print(f"  Max yield loss:     {check_df['yield_loss_pct'].max()*100:.1f}%")
print(f"  Min yield loss:     {check_df['yield_loss_pct'].min()*100:.1f}%")

print(f"\nRisk Bucket Distribution:")
print(check_df['risk_bucket'].value_counts().to_string())

print("\n" + "="*80)
print("Expected after fix:")
print("  - Average: 15-25% (not 40-50%)")
print("  - Most regions: Low or Moderate Risk (not High/Extreme)")
print("  - Max: 50-70% for Guatemala (not 95%)")
print("="*80 + "\n")



YIELD LOSS VALIDATION
Checking if yield losses are reasonable after elevation correction:

SSP2-4.5, 2050 - Yield Loss Distribution:
  Average yield loss: 5.1%
  Median yield loss:  3.4%
  Max yield loss:     17.9%
  Min yield loss:     0.7%

Risk Bucket Distribution:
risk_bucket
Low Risk (<20%)    28

Expected after fix:
  - Average: 15-25% (not 40-50%)
  - Most regions: Low or Moderate Risk (not High/Extreme)
  - Max: 50-70% for Guatemala (not 95%)



In [37]:
# ============================================================================
# STEP 6 (UPGRADED): FINANCIAL IMPACTS (Elasticity + Gross Margin + TCFD Ready)
# ============================================================================

print("\nSTEP 6: Calculating financial impacts (elasticity + gross margin)...")
print("-" * 70)

# ---------------------------------------------------------------
# Load financial parameters
# ---------------------------------------------------------------
coffee_price_per_kg = CONFIG['financial_params']['coffee_price_baseline_usd_kg']
coffee_revenue = CONFIG['financial_params']['coffee_revenue']
total_volume_mt = CONFIG['financial_params']['starbucks_annual_coffee_mt']
elasticity = CONFIG['financial_params']['price_elasticity']
gross_margin_pct = CONFIG['financial_params']['gross_margin_pct']

# ---------------------------------------------------------------
# 1. Baseline revenue & gross margin metrics
# ---------------------------------------------------------------
revenue_per_kg = coffee_revenue / (total_volume_mt * 1000)
gross_margin_per_kg = revenue_per_kg * gross_margin_pct
implied_markup = revenue_per_kg / coffee_price_per_kg

# ---------------------------------------------------------------
# 2. Supply shock and elasticity adjustment
# ---------------------------------------------------------------
yield_df['supply_shock_pct'] = yield_df['volume_at_risk_mt'] / total_volume_mt

# Elasticity formula: %ΔP = %ΔQ / elasticity
# Using point elasticity: %ΔP = -(elasticity * supply_shock)
yield_df['price_increase_pct'] = -elasticity * yield_df['supply_shock_pct']

yield_df['elasticity_multiplier'] = 1 + yield_df['price_increase_pct']

yield_df['adjusted_coffee_price_usd_kg'] = (
    coffee_price_per_kg * yield_df['elasticity_multiplier']
)

# ---------------------------------------------------------------
# 3. Cost impacts
# ---------------------------------------------------------------
yield_df['cost_green_coffee_usd'] = (
    yield_df['volume_at_risk_mt'] * 1000 * coffee_price_per_kg
)

yield_df['cost_green_coffee_adjusted_usd'] = (
    yield_df['volume_at_risk_mt'] * 1000 * yield_df['adjusted_coffee_price_usd_kg']
)

# ---------------------------------------------------------------
# 4. Profit impact (gross-margin method)
# ---------------------------------------------------------------
yield_df['profit_at_risk_usd'] = (
    yield_df['volume_at_risk_mt'] * 1000 * gross_margin_per_kg
)

# ---------------------------------------------------------------
# 5. Convert to millions for reporting
# ---------------------------------------------------------------
yield_df['cost_green_M'] = yield_df['cost_green_coffee_usd'] / 1_000_000
yield_df['cost_green_adjusted_M'] = yield_df['cost_green_coffee_adjusted_usd'] / 1_000_000
yield_df['profit_at_risk_M'] = yield_df['profit_at_risk_usd'] / 1_000_000

yield_df['pct_of_total_volume'] = (
    yield_df['volume_at_risk_mt'] / total_volume_mt * 100
)

# ---------------------------------------------------------------
# Diagnostics
# ---------------------------------------------------------------
print(f"✓ Baseline green coffee price: ${coffee_price_per_kg:.2f}/kg")
print(f"✓ Revenue per kg: ${revenue_per_kg:.2f}/kg")
print(f"✓ Gross margin per kg: ${gross_margin_per_kg:.2f}/kg")
print(f"✓ Gross margin percent: {gross_margin_pct*100:.1f}%")
print(f"✓ Implied markup: {implied_markup:.1f}x")
print(f"✓ Elasticity used: {elasticity}")
print("✓ Financial impacts calculated (baseline + elasticity + gross-margin profit)\n")



STEP 6: Calculating financial impacts (elasticity + gross margin)...
----------------------------------------------------------------------
✓ Baseline green coffee price: $3.50/kg
✓ Revenue per kg: $73.66/kg
✓ Gross margin per kg: $20.63/kg
✓ Gross margin percent: 28.0%
✓ Implied markup: 21.0x
✓ Elasticity used: -0.8
✓ Financial impacts calculated (baseline + elasticity + gross-margin profit)



add validation to confirm financial impacts are reasonable:

In [38]:
print("="*80)
print("FINANCIAL IMPACT - TOP 10 MOST VULNERABLE (SSP245, 2050)")
print("="*80)
print(f"{'Region':<40s} {'Yield Loss':<12s} {'Profit Risk':<15s} {'Price Impact':<15s}")
print("-"*82)

top_vulnerable = yield_df[
    (yield_df['scenario'] == 'ssp245') &
    (yield_df['period'] == 'mid_term')
].nlargest(10, 'profit_at_risk_M')

for _, row in top_vulnerable.iterrows():
    print(f"{row['region_name']:<40s} {row['yield_loss_pct']*100:>10.1f}% {row['profit_at_risk_M']:>13.0f}M {row['price_increase_pct']*100:>13.1f}%")

print("\n" + "="*80)
print("STARBUCKS vs COMPETITORS - FINANCIAL COMPARISON (SSP245, 2050)")
print("="*80)

starbucks_regions = ['Huila', 'Yirgacheffe', 'Sidamo', 'Central Highlands']
competitor_regions = ['Minas Gerais Sul', 'São Paulo', 'Espírito Santo']

ssp245_2050 = yield_df[
    (yield_df['scenario'] == 'ssp245') &
    (yield_df['period'] == 'mid_term')
]

print("\nStarbucks sources:")
sbux_data = ssp245_2050[ssp245_2050['region_name'].isin(starbucks_regions)]
for _, row in sbux_data.iterrows():
    print(f"  {row['region_name']:<30s}: {row['profit_at_risk_M']:>8.0f}M profit at risk ({row['yield_loss_pct']*100:>4.1f}% loss)")

sbux_total = sbux_data['profit_at_risk_M'].sum()
print(f"  {'TOTAL':<30s}: {sbux_total:>8.0f}M")

print("\nCompetitor sources:")
comp_data = ssp245_2050[ssp245_2050['region_name'].isin(competitor_regions)]
for _, row in comp_data.iterrows():
    print(f"  {row['region_name']:<30s}: {row['profit_at_risk_M']:>8.0f}M profit at risk ({row['yield_loss_pct']*100:>4.1f}% loss)")

comp_total = comp_data['profit_at_risk_M'].sum()
print(f"  {'TOTAL':<30s}: {comp_total:>8.0f}M")

print(f"\nCompetitive advantage: {comp_total - sbux_total:.0f}M in protected profit")
print("="*80)

FINANCIAL IMPACT - TOP 10 MOST VULNERABLE (SSP245, 2050)
Region                                   Yield Loss   Profit Risk     Price Impact   
----------------------------------------------------------------------------------
Sumatra Aceh                                   17.9%            25M           0.6%
Tarrazú                                        13.1%            19M           0.4%
Minas Gerais Sul                                4.9%            14M           0.3%
Antioquia                                       6.7%            13M           0.3%
São Paulo                                       7.8%            11M           0.3%
Central Highlands                              17.5%             9M           0.2%
Java West                                      11.4%             8M           0.2%
Espírito Santo                                  6.8%             7M           0.2%
West Valley                                     4.8%             6M           0.1%
Yunnan Pu er               

In [39]:
# ---------------------------------------------------------------
# Diagnostics
# ---------------------------------------------------------------
print(f"✓ Baseline green coffee price: ${coffee_price_per_kg:.2f}/kg")
print(f"✓ Revenue per kg: ${revenue_per_kg:.2f}/kg")
print(f"✓ Gross margin per kg: ${gross_margin_per_kg:.2f}/kg")
print(f"✓ Gross margin percent: {gross_margin_pct*100:.1f}%")
print(f"✓ Implied markup: {implied_markup:.1f}x")
print(f"✓ Elasticity used: {elasticity}")
print("✓ Financial impacts calculated (baseline + elasticity + gross-margin profit)\n")

# ⭐ ADD THIS VALIDATION:
print("="*80)
print("FINANCIAL IMPACT VALIDATION (SSP2-4.5, 2050)")
print("="*80)

ssp245_2050 = yield_df[(yield_df['scenario'] == 'ssp245') & (yield_df['period'] == 'mid_term')]

total_volume_at_risk = ssp245_2050['volume_at_risk_mt'].sum()
total_profit_at_risk = ssp245_2050['profit_at_risk_M'].sum()
pct_volume = (total_volume_at_risk / total_volume_mt) * 100

print(f"\nTotal Volume at Risk: {total_volume_at_risk:,.0f} MT ({pct_volume:.1f}% of supply)")
print(f"Total Profit at Risk: ${total_profit_at_risk:,.0f}M")
print(f"As % of Coffee Revenue: {(total_profit_at_risk*1e6 / coffee_revenue)*100:.1f}%")
print(f"As % of Total Assets: {(total_profit_at_risk*1e6 / CONFIG['financial_params']['total_assets'])*100:.2f}%")

# Top 5 regions by profit at risk
print(f"\nTop 5 Regions by Profit at Risk:")
top5 = ssp245_2050.nlargest(5, 'profit_at_risk_M')
for _, row in top5.iterrows():
    print(f"  {row['country']}_{row['region_name']:25s}: ${row['profit_at_risk_M']:6.1f}M " +
          f"({row['volume_at_risk_mt']:6,.0f} MT, {row['yield_loss_pct']*100:4.1f}% loss)")

print("\n" + "="*80)
print("EXPECTED AFTER ELEVATION FIX:")
print("  Total profit at risk: $500-600M (not $1,100M+)")
print("  Volume at risk: 14-16% of supply (not 30%+)")
print("  Top regions: $50-100M each (not $100-200M)")
print("="*80 + "\n")


✓ Baseline green coffee price: $3.50/kg
✓ Revenue per kg: $73.66/kg
✓ Gross margin per kg: $20.63/kg
✓ Gross margin percent: 28.0%
✓ Implied markup: 21.0x
✓ Elasticity used: -0.8
✓ Financial impacts calculated (baseline + elasticity + gross-margin profit)

FINANCIAL IMPACT VALIDATION (SSP2-4.5, 2050)

Total Volume at Risk: 7,701 MT (4.5% of supply)
Total Profit at Risk: $159M
As % of Coffee Revenue: 1.3%
As % of Total Assets: 0.45%

Top 5 Regions by Profit at Risk:
  Indonesia_Sumatra Aceh             : $  25.4M ( 1,232 MT, 17.9% loss)
  Costa Rica_Tarrazú                  : $  18.6M (   902 MT, 13.1% loss)
  Brazil_Minas Gerais Sul         : $  13.9M (   675 MT,  4.9% loss)
  Colombia_Antioquia                : $  13.0M (   631 MT,  6.7% loss)
  Brazil_São Paulo                : $  11.1M (   538 MT,  7.8% loss)

EXPECTED AFTER ELEVATION FIX:
  Total profit at risk: $500-600M (not $1,100M+)
  Volume at risk: 14-16% of supply (not 30%+)
  Top regions: $50-100M each (not $100-200M)



Just adding this below to see the pickel file

In [42]:
# ============================================================================
# STEP 7: AGGREGATE BY SCENARIO AND TIME PERIOD (TCFD-COMPLIANT)
# ============================================================================

import os
import pickle

print("STEP 7: Aggregating results by scenario...")
print("-"*70)

# ============================================================================
# 1. AGGREGATE ALL FINANCIAL + VOLUME METRICS
# ============================================================================

scenario_summary = (
    yield_df.groupby(["scenario", "period"])
    .agg({
        "volume_at_risk_mt": "sum",
        "cost_green_M": "sum",
        "cost_green_adjusted_M": "sum",
        "profit_at_risk_M": "sum",
    })
    .reset_index()
)

# ============================================================================
# 2. CORRECT % OF TOTAL SUPPLY AT RISK
# ============================================================================

scenario_summary["pct_at_risk"] = (
    scenario_summary["volume_at_risk_mt"] / total_volume_mt * 100
)

# ============================================================================
# 3. ADD HUMAN-READABLE SCENARIO / YEAR LABELS
# ============================================================================

scenario_names = {
    "ssp126": "Net Zero 2050",
    "ssp245": "Delayed Transition",
    "ssp585": "Current Policies",
}

period_names = {
    "near_term": "2030",
    "mid_term": "2050",
}

scenario_summary["scenario_name"] = scenario_summary["scenario"].map(scenario_names)
scenario_summary["year"] = scenario_summary["period"].map(period_names)

print("✓ Aggregation complete\n")

# ============================================================================
# DISPLAY RESULTS
# ============================================================================

print("="*70)
print("KEY RESULTS: COFFEE SUPPLY AT RISK & FINANCIAL IMPACT")
print("="*70)

for scenario in ["ssp126", "ssp245", "ssp585"]:

    data = scenario_summary[scenario_summary["scenario"] == scenario]
    if len(data) == 0:
        continue

    print(f"\n{scenario_names[scenario]}")

    for _, row in data.iterrows():
        print(f"  {row['year']}:")
        print(f"      Volume at risk:            {row['volume_at_risk_mt']:,.0f} MT")
        print(f"      % of total supply:         {row['pct_at_risk']:.2f}%")
        print(f"      Baseline procurement cost: ${row['cost_green_M']:.1f}M")
        print(f"      Elasticity-adjusted cost:  ${row['cost_green_adjusted_M']:.1f}M")
        print(f"      Profit at risk:            ${row['profit_at_risk_M']:.1f}M")

print("\n" + "="*70 + "\n")

# ============================================================================
# SAVE OUTPUTS
# ============================================================================

yield_df.to_csv(f"{CONFIG['base_path']}/outputs/yield_financial_impacts.csv", index=False)
scenario_summary.to_csv(f"{CONFIG['base_path']}/outputs/scenario_summary.csv", index=False)

output_dir = f"{CONFIG['base_path']}/outputs/intermediate"
os.makedirs(output_dir, exist_ok=True)

with open(f"{output_dir}/financial_impacts.pkl", "wb") as f:
    pickle.dump(
        {
            "detailed": yield_df,
            "summary": scenario_summary,
            "parameters": {
                "coffee_price_per_kg": coffee_price_per_kg,
                "revenue_per_kg": revenue_per_kg,
                "implied_markup": implied_markup,
                "total_volume_mt": total_volume_mt,
            },
        },
        f,
    )

print("✓ Saved financial impact results to outputs/")
print("  - yield_financial_impacts.csv (detailed)")
print("  - scenario_summary.csv (aggregated)")
print("  - financial_impacts.pkl (for next modules)")


STEP 7: Aggregating results by scenario...
----------------------------------------------------------------------
✓ Aggregation complete

KEY RESULTS: COFFEE SUPPLY AT RISK & FINANCIAL IMPACT

Net Zero 2050
  2050:
      Volume at risk:            5,733 MT
      % of total supply:         3.33%
      Baseline procurement cost: $20.1M
      Elasticity-adjusted cost:  $20.1M
      Profit at risk:            $118.2M
  2030:
      Volume at risk:            5,564 MT
      % of total supply:         3.23%
      Baseline procurement cost: $19.5M
      Elasticity-adjusted cost:  $19.5M
      Profit at risk:            $114.8M

Delayed Transition
  2050:
      Volume at risk:            7,701 MT
      % of total supply:         4.48%
      Baseline procurement cost: $27.0M
      Elasticity-adjusted cost:  $27.0M
      Profit at risk:            $158.8M
  2030:
      Volume at risk:            7,200 MT
      % of total supply:         4.19%
      Baseline procurement cost: $25.2M
      Elastici

In [ ]:
# ============================================================================
# STEP 7: AGGREGATE BY SCENARIO AND TIME PERIOD (TCFD-COMPLIANT)
# ============================================================================

import os
import pickle

print("STEP 7: Aggregating results by scenario...")
print("-"*70)

# ============================================================================
# 1. AGGREGATE ALL FINANCIAL + VOLUME METRICS
# ============================================================================

scenario_summary = (
    yield_df.groupby(["scenario", "period"])
    .agg({
        "volume_at_risk_mt": "sum",
        "cost_green_M": "sum",
        "cost_green_adjusted_M": "sum",
        "profit_at_risk_M": "sum",
    })
    .reset_index()
)

# ============================================================================
# 2. CORRECT % OF TOTAL SUPPLY AT RISK
# ============================================================================

scenario_summary["pct_at_risk"] = (
    scenario_summary["volume_at_risk_mt"] / total_volume_mt * 100
)

# ============================================================================
# 3. ADD HUMAN-READABLE SCENARIO / YEAR LABELS
# ============================================================================

scenario_names = {
    "ssp126": "Net Zero 2050",
    "ssp245": "Delayed Transition",
    "ssp585": "Current Policies",
}

period_names = {
    "near_term": "2030",
    "mid_term": "2050",
}

scenario_summary["scenario_name"] = scenario_summary["scenario"].map(scenario_names)
scenario_summary["year"] = scenario_summary["period"].map(period_names)

print("✓ Aggregation complete\n")

# ============================================================================
# DISPLAY RESULTS
# ============================================================================

print("="*70)
print("KEY RESULTS: COFFEE SUPPLY AT RISK & FINANCIAL IMPACT")
print("="*70)

for scenario in ["ssp126", "ssp245", "ssp585"]:

    data = scenario_summary[scenario_summary["scenario"] == scenario]
    if len(data) == 0:
        continue

    print(f"\n{scenario_names[scenario]}")

    for _, row in data.iterrows():
        print(f"  {row['year']}:")
        print(f"      Volume at risk:            {row['volume_at_risk_mt']:,.0f} MT")
        print(f"      % of total supply:         {row['pct_at_risk']:.2f}%")
        print(f"      Baseline procurement cost: ${row['cost_green_M']:.1f}M")
        print(f"      Elasticity-adjusted cost:  ${row['cost_green_adjusted_M']:.1f}M")
        print(f"      Profit at risk:            ${row['profit_at_risk_M']:.1f}M")

print("\n" + "="*70 + "\n")

# ============================================================================
# SAVE OUTPUTS
# ============================================================================

yield_df.to_csv(f"{CONFIG['base_path']}/outputs/yield_financial_impacts.csv", index=False)
scenario_summary.to_csv(f"{CONFIG['base_path']}/outputs/scenario_summary.csv", index=False)

output_dir = f"{CONFIG['base_path']}/outputs/intermediate"
os.makedirs(output_dir, exist_ok=True)

with open(f"{output_dir}/financial_impacts.pkl", "wb") as f:
    pickle.dump(
        {
            "detailed": yield_df,
            "summary": scenario_summary,
            "parameters": {
                "coffee_price_per_kg": coffee_price_per_kg,
                "revenue_per_kg": revenue_per_kg,
                "implied_markup": implied_markup,
                "total_volume_mt": total_volume_mt,
            },
        },
        f,
    )

print("✓ Saved financial impact results to outputs/")
print("  - yield_financial_impacts.csv (detailed)")
print("  - scenario_summary.csv (aggregated)")
print("  - financial_impacts.pkl (for next modules)")


STEP 7: Aggregating results by scenario...
----------------------------------------------------------------------
✓ Aggregation complete

KEY RESULTS: COFFEE SUPPLY AT RISK & FINANCIAL IMPACT

Net Zero 2050
  2050:
      Volume at risk:            5,733 MT
      % of total supply:         3.33%
      Baseline procurement cost: $20.1M
      Elasticity-adjusted cost:  $20.1M
      Profit at risk:            $118.2M
  2030:
      Volume at risk:            5,564 MT
      % of total supply:         3.23%
      Baseline procurement cost: $19.5M
      Elasticity-adjusted cost:  $19.5M
      Profit at risk:            $114.8M

Delayed Transition
  2050:
      Volume at risk:            7,701 MT
      % of total supply:         4.48%
      Baseline procurement cost: $27.0M
      Elasticity-adjusted cost:  $27.0M
      Profit at risk:            $158.8M
  2030:
      Volume at risk:            7,200 MT
      % of total supply:         4.19%
      Baseline procurement cost: $25.2M
      Elastici

# ============================================================================
# STEP 7: AGGREGATE BY SCENARIO AND TIME PERIOD (TCFD-COMPLIANT)
# ============================================================================
import os
import pickle
print("STEP 7: Aggregating results by scenario...")
print("-"*70)

scenario_summary = (
    yield_df.groupby(["scenario", "period"])
    .agg({
        "volume_at_risk_mt": "sum",
        "cost_green_M": "sum",
        "cost_green_adjusted_M": "sum",
        "profit_at_risk_M": "sum",
    })
    .reset_index()
)

scenario_summary["pct_at_risk"] = (
    scenario_summary["volume_at_risk_mt"] / total_volume_mt * 100
)

scenario_names = {
    "ssp126": "Net Zero 2050",
    "ssp245": "Delayed Transition",
    "ssp585": "Current Policies",
}
period_names = {
    "near_term": "2030",
    "mid_term": "2050",
}
scenario_summary["scenario_name"] = scenario_summary["scenario"].map(scenario_names)
scenario_summary["year"] = scenario_summary["period"].map(period_names)
print("✓ Aggregation complete\n")

print("="*70)
print("KEY RESULTS: COFFEE SUPPLY AT RISK & FINANCIAL IMPACT")
print("="*70)
for scenario in ["ssp126", "ssp245", "ssp585"]:
    data = scenario_summary[scenario_summary["scenario"] == scenario]
    if len(data) == 0:
        continue
    print(f"\n{scenario_names[scenario]}")
    for _, row in data.iterrows():
        print(f"  {row['year']}:")
        print(f"      Volume at risk:            {row['volume_at_risk_mt']:,.0f} MT")
        print(f"      % of total supply:         {row['pct_at_risk']:.2f}%")
        print(f"      Baseline procurement cost: ${row['cost_green_M']:.1f}M")
        print(f"      Elasticity-adjusted cost:  ${row['cost_green_adjusted_M']:.1f}M")
        print(f"      Profit at risk:            ${row['profit_at_risk_M']:.1f}M")
print("\n" + "="*70 + "\n")

In [43]:
#Now we need to convert these annual impacts into Net Present Value (NPV) over 30 years. That's the total financial exposure figure.
print("\n" + "="*80)
print("NET PRESENT VALUE CALCULATION (30-YEAR HORIZON)")
print("="*80)

discount_rate = CONFIG['financial_params']['discount_rate']

# Calculate NPV for each scenario/period
npv_results = []

for scenario in ["ssp126", "ssp245", "ssp585"]:
    scenario_data = scenario_summary[scenario_summary["scenario"] == scenario]
    
    # Mid-term (2050) represents 2041-2060, use as proxy for 2030-2060
    mid_term_data = scenario_data[scenario_data["period"] == "mid_term"]
    
    if len(mid_term_data) > 0:
        annual_profit_risk = mid_term_data.iloc[0]["profit_at_risk_M"]
        
        # NPV: PV = FV / (1 + r)^t summed over 30 years
        npv = sum([annual_profit_risk / ((1 + discount_rate) ** t) for t in range(1, 31)])
        
        scenario_name = scenario_names[scenario]
        print(f"\n{scenario_name}:")
        print(f"  Annual profit at risk (2050): ${annual_profit_risk:.1f}M")
        print(f"  NPV (30-year, 8% discount):  ${npv:.1f}M")
        
        npv_results.append({
            "scenario": scenario,
            "scenario_name": scenario_name,
            "annual_impact": annual_profit_risk,
            "npv_30yr": npv
        })

npv_df = pd.DataFrame(npv_results)

print("\n" + "="*80)
print("NPV SUMMARY")
print("="*80)
print(f"{'Scenario':<30s} {'Annual':<15s} {'30-Yr NPV':<15s}")
print("-"*60)
for _, row in npv_df.iterrows():
    print(f"{row['scenario_name']:<30s} ${row['annual_impact']:>12.1f}M ${row['npv_30yr']:>12.1f}M")

print("="*80)
# Save NPV results
npv_df.to_csv(f"{CONFIG['base_path']}/outputs/npv_summary.csv", index=False)

print(f"\n✓ Saved NPV results to: {CONFIG['base_path']}/outputs/npv_summary.csv")


NET PRESENT VALUE CALCULATION (30-YEAR HORIZON)

Net Zero 2050:
  Annual profit at risk (2050): $118.2M
  NPV (30-year, 8% discount):  $1331.2M

Delayed Transition:
  Annual profit at risk (2050): $158.8M
  NPV (30-year, 8% discount):  $1788.1M

Current Policies:
  Annual profit at risk (2050): $175.4M
  NPV (30-year, 8% discount):  $1974.5M

NPV SUMMARY
Scenario                       Annual          30-Yr NPV      
------------------------------------------------------------
Net Zero 2050                  $       118.2M $      1331.2M
Delayed Transition             $       158.8M $      1788.1M
Current Policies               $       175.4M $      1974.5M

✓ Saved NPV results to: C:/Users/ibeha/OneDrive/Desktop/Climate/outputs/npv_summary.csv


In [44]:
# Add NPV to scenario_summary
scenario_summary_with_npv = scenario_summary.copy()

for idx, row in scenario_summary_with_npv.iterrows():
    if row['period'] == 'mid_term':
        scenario = row['scenario']
        npv_value = npv_df[npv_df['scenario'] == scenario]['npv_30yr'].values[0]
        scenario_summary_with_npv.loc[idx, 'npv_30yr_M'] = npv_value
    else:
        scenario_summary_with_npv.loc[idx, 'npv_30yr_M'] = np.nan

# Save updated summary
scenario_summary_with_npv.to_csv(f"{CONFIG['base_path']}/outputs/scenario_summary_with_npv.csv", index=False)

print(f"✓ Saved scenario summary with NPV to: {CONFIG['base_path']}/outputs/scenario_summary_with_npv.csv")

✓ Saved scenario summary with NPV to: C:/Users/ibeha/OneDrive/Desktop/Climate/outputs/scenario_summary_with_npv.csv


In [45]:
# ============================================================================
# STEP 8: CARBON PRICING & TRANSITION COSTS (FIXED - ADD CODE COLUMN)
# ============================================================================
print("\nSTEP 8: Calculating carbon pricing & transition costs...")
print("-"*70)

# Emissions parameters
scope_1_2_emissions = CONFIG['financial_params']['scope_1_2_emissions']
scope_3_emissions = CONFIG['financial_params']['scope_3_emissions']
scope_3_agriculture = scope_3_emissions * 0.40

print("Starbucks Emissions Baseline:")
print(f"  Scope 1+2: {scope_1_2_emissions/1e6:.2f}M MT")
print(f"  Scope 3 Total: {scope_3_emissions/1e6:.2f}M MT")
print(f"  Agriculture portion: {scope_3_agriculture/1e6:.2f}M MT")
print(f"  Relevant emissions: {(scope_1_2_emissions+scope_3_agriculture)/1e6:.2f}M MT\n")

# ⭐ MAPPING: Excel names → scenario codes
scenario_name_to_code = {
    "Net Zero 2050": "ssp126",
    "Net-Zero 2050": "ssp126",
    "Delayed Transition": "ssp245",
    "Current Policies": "ssp585"
}

carbon_cost_rows = []

for _, row in carbon_prices.iterrows():
    scenario_name = row["scenario"]  # From Excel (e.g., "Net Zero 2050")
    year = row["year"]
    carbon_price = row["carbon_price_usd_per_tco2"]
    
    # ⭐ MAP NAME TO CODE
    scenario_code = scenario_name_to_code.get(scenario_name, scenario_name)
    
    coverage = row.get("policy_coverage_pct", 100) / 100
    passthrough = row.get("supplier_passthrough_rate", 0)
    
    scope12_cost = scope_1_2_emissions * carbon_price * coverage
    scope3_cost = scope_3_agriculture * carbon_price * coverage * passthrough
    total_cost = scope12_cost + scope3_cost
    
    carbon_cost_rows.append({
        "scenario": scenario_code,  # ⭐ NOW STORES CODE (ssp126, etc.)
        "scenario_name": scenario_name,  # Keep original name for display
        "year": year,
        "carbon_price_usd": carbon_price,
        "coverage_pct": coverage * 100,
        "passthrough_rate": passthrough,
        "scope_1_2_cost_M": scope12_cost / 1e6,
        "scope_3_cost_M": scope3_cost / 1e6,
        "total_carbon_cost_M": total_cost / 1e6,
        "pct_of_revenue": total_cost / CONFIG["financial_params"]["total_revenue"] * 100
    })

carbon_costs_df = pd.DataFrame(carbon_cost_rows)
print("✓ Carbon costs calculated successfully\n")

# Display results
scenario_display_map = {
    "ssp126": "Net Zero 2050",
    "ssp245": "Delayed Transition",
    "ssp585": "Current Policies"
}

print("="*70)
print("CARBON PRICING IMPACT BY SCENARIO")
print("="*70)

for scenario_code, scenario_name in scenario_display_map.items():
    subset = carbon_costs_df[carbon_costs_df["scenario"] == scenario_code]
    if subset.empty:
        continue
    
    print(f"\n{scenario_name} ({scenario_code})")
    for _, r in subset.iterrows():
        print(f"  {int(r['year'])}:")
        print(f"      Carbon price: ${r['carbon_price_usd']:.0f}/tCO2")
        print(f"      Annual carbon cost: ${r['total_carbon_cost_M']:.1f}M "
              f"({r['pct_of_revenue']:.2f}% of revenue)")
        print(f"      Scope 1+2 cost:  ${r['scope_1_2_cost_M']:.1f}M")
        print(f"      Scope 3 cost:    ${r['scope_3_cost_M']:.1f}M")

print("\n" + "="*70)


STEP 8: Calculating carbon pricing & transition costs...
----------------------------------------------------------------------
Starbucks Emissions Baseline:
  Scope 1+2: 1.50M MT
  Scope 3 Total: 15.00M MT
  Agriculture portion: 6.00M MT
  Relevant emissions: 7.50M MT

✓ Carbon costs calculated successfully

CARBON PRICING IMPACT BY SCENARIO

Net Zero 2050 (ssp126)
  2025:
      Carbon price: $40/tCO2
      Annual carbon cost: $42.0M (0.12% of revenue)
      Scope 1+2 cost:  $21.0M
      Scope 3 cost:    $21.0M
  2030:
      Carbon price: $100/tCO2
      Annual carbon cost: $216.0M (0.60% of revenue)
      Scope 1+2 cost:  $90.0M
      Scope 3 cost:    $126.0M
  2035:
      Carbon price: $150/tCO2
      Annual carbon cost: $438.8M (1.21% of revenue)
      Scope 1+2 cost:  $168.8M
      Scope 3 cost:    $270.0M
  2040:
      Carbon price: $200/tCO2
      Annual carbon cost: $663.0M (1.83% of revenue)
      Scope 1+2 cost:  $255.0M
      Scope 3 cost:    $408.0M
  2045:
      Carbon pr

In [46]:
# Save carbon costs
carbon_costs_df.to_csv(f"{CONFIG['base_path']}/outputs/carbon_pricing_costs.csv", index=False)

print(f"\n✓ Saved carbon pricing results to: {CONFIG['base_path']}/outputs/carbon_pricing_costs.csv")

# Summary
print("\n" + "="*80)
print("CARBON COST SUMMARY - 2050")
print("="*80)
for scenario_code in ['ssp126', 'ssp245', 'ssp585']:
    subset = carbon_costs_df[(carbon_costs_df['scenario'] == scenario_code) & (carbon_costs_df['year'] == 2050)]
    if not subset.empty:
        cost = subset.iloc[0]['total_carbon_cost_M']
        pct_rev = subset.iloc[0]['pct_of_revenue']
        scenario_name = scenario_display_map[scenario_code]
        print(f"{scenario_name:<30s}: ${cost:>10.1f}M ({pct_rev:>5.2f}% of revenue)")

print("="*80)


✓ Saved carbon pricing results to: C:/Users/ibeha/OneDrive/Desktop/Climate/outputs/carbon_pricing_costs.csv

CARBON COST SUMMARY - 2050
Net Zero 2050                 : $     877.5M ( 2.42% of revenue)
Delayed Transition            : $    2295.0M ( 6.34% of revenue)
Current Policies              : $      32.4M ( 0.09% of revenue)


In [47]:
print("="*80)
print("DIAGNOSTIC: CHECKING DATA AVAILABILITY")
print("="*80)

print("\n1. PHYSICAL RISKS (scenario_summary):")
print(scenario_summary[['scenario', 'year', 'profit_at_risk_M']])

print("\n2. CARBON COSTS (carbon_costs_df):")
print(carbon_costs_df[['scenario', 'year', 'total_carbon_cost_M']])

print("\n3. CHECKING MATCHES:")
for code in ['ssp126', 'ssp245', 'ssp585']:
    print(f"\n{code}:")
    
    phys_2030 = scenario_summary[(scenario_summary['scenario'] == code) & (scenario_summary['year'] == 2030)]
    phys_2050 = scenario_summary[(scenario_summary['scenario'] == code) & (scenario_summary['year'] == 2050)]
    carb_2030 = carbon_costs_df[(carbon_costs_df['scenario'] == code) & (carbon_costs_df['year'] == 2030)]
    carb_2050 = carbon_costs_df[(carbon_costs_df['scenario'] == code) & (carbon_costs_df['year'] == 2050)]
    
    print(f"  Physical 2030: {'Found' if not phys_2030.empty else 'MISSING'}")
    print(f"  Physical 2050: {'Found' if not phys_2050.empty else 'MISSING'}")
    print(f"  Carbon 2030: {'Found' if not carb_2030.empty else 'MISSING'}")
    print(f"  Carbon 2050: {'Found' if not carb_2050.empty else 'MISSING'}")

print("="*80)

DIAGNOSTIC: CHECKING DATA AVAILABILITY

1. PHYSICAL RISKS (scenario_summary):
  scenario  year  profit_at_risk_M
0   ssp126  2050        118.247340
1   ssp126  2030        114.763369
2   ssp245  2050        158.833810
3   ssp245  2030        148.496078
4   ssp585  2050        175.388033
5   ssp585  2030        147.921496

2. CARBON COSTS (carbon_costs_df):
   scenario  year  total_carbon_cost_M
0    ssp126  2025               42.000
1    ssp126  2030              216.000
2    ssp126  2035              438.750
3    ssp126  2040              663.000
4    ssp126  2045              772.200
5    ssp126  2050              877.500
6    ssp245  2025               13.500
7    ssp245  2030              126.000
8    ssp245  2035             1701.000
9    ssp245  2040             3391.500
10   ssp245  2045             2851.200
11   ssp245  2050             2295.000
12   ssp585  2025                7.200
13   ssp585  2030               10.560
14   ssp585  2035               15.000
15   ssp585  2040

In [48]:
# ============================================================================
# STEP 9: COMBINE PHYSICAL + TRANSITION RISKS (FIXED)
# ============================================================================
print("\nSTEP 9: Combining physical and transition risks...")
print("-"*70)

combined_impacts = []

scenario_map = {
    'ssp126': 'Net Zero 2050',
    'ssp245': 'Delayed Transition',
    'ssp585': 'Current Policies'
}

# Ensure year is int
scenario_summary['year'] = scenario_summary['year'].astype(int)
carbon_costs_df['year'] = carbon_costs_df['year'].astype(int)

for code, name in scenario_map.items():
    # Get physical risks (uses CODE)
    phys_2030 = scenario_summary[
        (scenario_summary['scenario'] == code) &
        (scenario_summary['year'] == 2030)
    ]
    phys_2050 = scenario_summary[
        (scenario_summary['scenario'] == code) &
        (scenario_summary['year'] == 2050)
    ]
    
    # Get carbon risks (ALSO uses CODE - THIS WAS THE BUG!)
    carb_2030 = carbon_costs_df[
        (carbon_costs_df['scenario'] == code) &  # ← CHANGED from 'name' to 'code'
        (carbon_costs_df['year'] == 2030)
    ]
    carb_2050 = carbon_costs_df[
        (carbon_costs_df['scenario'] == code) &  # ← CHANGED from 'name' to 'code'
        (carbon_costs_df['year'] == 2050)
    ]
    
    # Combine 2030
    if not phys_2030.empty and not carb_2030.empty:
        combined_impacts.append({
            'scenario_code': code,
            'scenario_name': name,
            'year': 2030,
            'physical_risk_M': phys_2030['profit_at_risk_M'].values[0],
            'carbon_cost_M': carb_2030['total_carbon_cost_M'].values[0],
            'total_climate_cost_M': (phys_2030['profit_at_risk_M'].values[0] +
                                    carb_2030['total_carbon_cost_M'].values[0]),
            'volume_at_risk_mt': phys_2030['volume_at_risk_mt'].values[0],
            'pct_supply_at_risk': phys_2030['pct_at_risk'].values[0]
        })
    
    # Combine 2050
    if not phys_2050.empty and not carb_2050.empty:
        combined_impacts.append({
            'scenario_code': code,
            'scenario_name': name,
            'year': 2050,
            'physical_risk_M': phys_2050['profit_at_risk_M'].values[0],
            'carbon_cost_M': carb_2050['total_carbon_cost_M'].values[0],
            'total_climate_cost_M': (phys_2050['profit_at_risk_M'].values[0] +
                                    carb_2050['total_carbon_cost_M'].values[0]),
            'volume_at_risk_mt': phys_2050['volume_at_risk_mt'].values[0],
            'pct_supply_at_risk': phys_2050['pct_at_risk'].values[0]
        })

combined_df = pd.DataFrame(combined_impacts)

print("\n✓ Combined physical + transition risks")

# Display results
print("\n" + "="*80)
print("TOTAL CLIMATE RISK: PHYSICAL + TRANSITION (ANNUAL COSTS)")
print("="*80)

for scenario_code in ['ssp126', 'ssp245', 'ssp585']:
    subset = combined_df[combined_df['scenario_code'] == scenario_code]
    if subset.empty:
        continue
    
    scenario_name = scenario_map[scenario_code]
    print(f"\n{scenario_name} ({scenario_code})")
    
    for _, row in subset.iterrows():
        print(f"  {int(row['year'])}:")
        print(f"      Physical risks (supply):  ${row['physical_risk_M']:.1f}M")
        print(f"      Transition risks (carbon): ${row['carbon_cost_M']:.1f}M")
        print(f"      ─────────────────────────────────────")
        print(f"      TOTAL CLIMATE COST:       ${row['total_climate_cost_M']:.1f}M")
        print(f"      Volume at risk:           {row['volume_at_risk_mt']:,.0f} MT ({row['pct_supply_at_risk']:.1f}%)")

print("\n" + "="*80)

# Save outputs
combined_df.to_csv(f"{CONFIG['base_path']}/outputs/total_climate_costs.csv", index=False)
print("✓ Saved combined results to outputs/total_climate_costs.csv")


STEP 9: Combining physical and transition risks...
----------------------------------------------------------------------

✓ Combined physical + transition risks

TOTAL CLIMATE RISK: PHYSICAL + TRANSITION (ANNUAL COSTS)

Net Zero 2050 (ssp126)
  2030:
      Physical risks (supply):  $114.8M
      Transition risks (carbon): $216.0M
      ─────────────────────────────────────
      TOTAL CLIMATE COST:       $330.8M
      Volume at risk:           5,564 MT (3.2%)
  2050:
      Physical risks (supply):  $118.2M
      Transition risks (carbon): $877.5M
      ─────────────────────────────────────
      TOTAL CLIMATE COST:       $995.7M
      Volume at risk:           5,733 MT (3.3%)

Delayed Transition (ssp245)
  2030:
      Physical risks (supply):  $148.5M
      Transition risks (carbon): $126.0M
      ─────────────────────────────────────
      TOTAL CLIMATE COST:       $274.5M
      Volume at risk:           7,200 MT (4.2%)
  2050:
      Physical risks (supply):  $158.8M
      Transitio

In [49]:
print("\n" + "="*80)
print("TOTAL CLIMATE RISK - 30-YEAR NET PRESENT VALUE")
print("="*80)

discount_rate = CONFIG['financial_params']['discount_rate']

total_npv_results = []

for scenario_code in ['ssp126', 'ssp245', 'ssp585']:
    scenario_name = scenario_map[scenario_code]
    
    # Get 2050 data (represents mid-term onwards)
    mid_term = combined_df[
        (combined_df['scenario_code'] == scenario_code) &
        (combined_df['year'] == 2050)
    ]
    
    if not mid_term.empty:
        annual_cost = mid_term.iloc[0]['total_climate_cost_M']
        
        # NPV over 30 years
        npv = sum([annual_cost / ((1 + discount_rate) ** t) for t in range(1, 31)])
        
        total_npv_results.append({
            'scenario': scenario_code,
            'scenario_name': scenario_name,
            'annual_cost_2050_M': annual_cost,
            'npv_30yr_M': npv,
            'pct_of_total_revenue': (npv / CONFIG['financial_params']['total_revenue']) * 100,
            'pct_of_total_assets': (npv / CONFIG['financial_params']['total_assets']) * 100
        })

total_npv_df = pd.DataFrame(total_npv_results)

print(f"\n{'Scenario':<30s} {'Annual 2050':<15s} {'30-Yr NPV':<15s} {'% Assets':<12s}")
print("-"*72)

for _, row in total_npv_df.iterrows():
    print(f"{row['scenario_name']:<30s} ${row['annual_cost_2050_M']:>12.1f}M ${row['npv_30yr_M']:>12.1f}M {row['pct_of_total_assets']:>10.2f}%")

print("\n" + "="*80)
print("MATERIALITY ASSESSMENT (TCFD)")
print("="*80)
print("\nFinancial materiality threshold: 5% of total assets")

for _, row in total_npv_df.iterrows():
    material = "✓ MATERIAL" if row['pct_of_total_assets'] > 5 else "✗ Not material"
    print(f"{row['scenario_name']:<30s}: {row['pct_of_total_assets']:>6.2f}% of assets - {material}")

print("="*80)

# Save
total_npv_df.to_csv(f"{CONFIG['base_path']}/outputs/total_climate_npv.csv", index=False)
print("\n✓ Saved to outputs/total_climate_npv.csv")


TOTAL CLIMATE RISK - 30-YEAR NET PRESENT VALUE

Scenario                       Annual 2050     30-Yr NPV       % Assets    
------------------------------------------------------------------------
Net Zero 2050                  $       995.7M $     11209.9M       0.00%
Delayed Transition             $      2453.8M $     27624.7M       0.00%
Current Policies               $       207.8M $      2339.2M       0.00%

MATERIALITY ASSESSMENT (TCFD)

Financial materiality threshold: 5% of total assets
Net Zero 2050                 :   0.00% of assets - ✗ Not material
Delayed Transition            :   0.00% of assets - ✗ Not material
Current Policies              :   0.00% of assets - ✗ Not material

✓ Saved to outputs/total_climate_npv.csv


In [50]:
combined_df.head()


,scenario_code,scenario_name,year,physical_risk_M,carbon_cost_M,total_climate_cost_M,volume_at_risk_mt,pct_supply_at_risk
0,ssp126,Net Zero 2050,2030,114.763369,216.00,330.763369,5564.127708,3.234958
1,ssp126,Net Zero 2050,2050,118.247340,877.50,995.747340,5733.042765,3.333164
2,ssp245,Delayed Transition,2030,148.496078,126.00,274.496078,7199.606913,4.185818
3,ssp245,Delayed Transition,2050,158.833810,2295.00,2453.833810,7700.816131,4.477219
4,ssp585,Current Policies,2030,147.921496,10.56,158.481496,7171.749175,4.169622


In [51]:
# ============================================================================
# STEP 10: REGIONAL RISK CONCENTRATION ANALYSIS
# ============================================================================

print("\n" + "="*80)
print("STEP 10: REGIONAL RISK CONCENTRATION ANALYSIS")
print("="*80)

# Worst case: SSP585, 2050
regional_risk = yield_df[
    (yield_df['scenario'] == 'ssp585') & 
    (yield_df['period'] == 'mid_term')
].copy()

# Sort by financial impact (profit at risk)
regional_risk = regional_risk.sort_values('profit_at_risk_M', ascending=False)

print("\nTop 10 Most Vulnerable Regions (2050, Current Policies):")
print("-" * 70)

for idx, row in regional_risk.head(10).iterrows():
    print(f"{row['region_name']}, {row['country']}")
    print(f"  Annual Volume: {row['annual_mt']:,.0f} MT ({row['sourcing_pct']:.1f}% of supply)")
    print(f"  Yield Loss: {row['yield_loss_pct']*100:.1f}%")
    print(f"  Volume at Risk: {row['volume_at_risk_mt']:,.0f} MT")
    print(f"  Profit at Risk: ${row['profit_at_risk_M']:.1f}M")
    print()

# Concentration metrics
regional_risk['cumulative_cost'] = regional_risk['profit_at_risk_M'].cumsum()
regional_risk['cumulative_pct'] = (
    regional_risk['cumulative_cost'] / regional_risk['profit_at_risk_M'].sum() * 100
)

top_3_cost = regional_risk['profit_at_risk_M'].head(3).sum()
top_5_cost = regional_risk['profit_at_risk_M'].head(5).sum()
top_10_cost = regional_risk['profit_at_risk_M'].head(10).sum()
total_cost = regional_risk['profit_at_risk_M'].sum()

top_3_volume = regional_risk['sourcing_pct'].head(3).sum()
top_5_volume = regional_risk['sourcing_pct'].head(5).sum()

print(f"CONCENTRATION RISK METRICS (2050, Current Policies):")
print("-" * 70)
print(f"  Top 3 regions: ${top_3_cost:.1f}M ({top_3_cost/total_cost*100:.1f}% of total risk)")
print(f"                 {top_3_volume:.1f}% of total supply volume")
print(f"  Top 5 regions: ${top_5_cost:.1f}M ({top_5_cost/total_cost*100:.1f}% of total risk)")
print(f"                 {top_5_volume:.1f}% of total supply volume")
print(f"  Top 10 regions: ${top_10_cost:.1f}M ({top_10_cost/total_cost*100:.1f}% of total risk)")
print()

# Risk classification
concentration_risk_level = (
    'CRITICAL' if top_3_cost/total_cost > 0.5 else
    'HIGH' if top_3_cost/total_cost > 0.4 else
    'MODERATE' if top_3_cost/total_cost > 0.25 else 
    'LOW'
)

print(f"  Geographic concentration risk: {concentration_risk_level}")
print(f"  Recommendation: {'URGENT diversification needed' if concentration_risk_level in ['CRITICAL', 'HIGH'] else 'Monitor and plan diversification'}")
print()

# Save results
regional_risk.to_csv(f"{CONFIG['base_path']}/outputs/10_regional_risk_concentration.csv", index=False)



STEP 10: REGIONAL RISK CONCENTRATION ANALYSIS

Top 10 Most Vulnerable Regions (2050, Current Policies):
----------------------------------------------------------------------
Sumatra Aceh, Indonesia
  Annual Volume: 6,900 MT (4.0% of supply)
  Yield Loss: 23.5%
  Volume at Risk: 1,625 MT
  Profit at Risk: $33.5M

Antioquia, Colombia
  Annual Volume: 9,400 MT (5.5% of supply)
  Yield Loss: 8.6%
  Volume at Risk: 809 MT
  Profit at Risk: $16.7M

Minas Gerais Sul, Brazil
  Annual Volume: 13,700 MT (8.0% of supply)
  Yield Loss: 5.3%
  Volume at Risk: 729 MT
  Profit at Risk: $15.0M

São Paulo, Brazil
  Annual Volume: 6,900 MT (4.0% of supply)
  Yield Loss: 8.2%
  Volume at Risk: 565 MT
  Profit at Risk: $11.7M

Central Highlands, Vietnam
  Annual Volume: 2,600 MT (1.5% of supply)
  Yield Loss: 19.5%
  Volume at Risk: 508 MT
  Profit at Risk: $10.5M

Java West, Indonesia
  Annual Volume: 3,400 MT (2.0% of supply)
  Yield Loss: 14.8%
  Volume at Risk: 503 MT
  Profit at Risk: $10.4M

Chiap

In [52]:
# ============================================================================
# STEP 11: SUPPLY CHAIN RESILIENCE & DIVERSIFICATION
# ============================================================================

print("\n" + "="*80)
print("STEP 11: SUPPLY CHAIN RESILIENCE & DIVERSIFICATION")
print("="*80)

# Use SSP585 (worst case) to categorize regions by risk
resilience_df = yield_df[
    (yield_df['scenario'] == 'ssp585') & 
    (yield_df['period'] == 'mid_term')
].copy()

# Create vulnerability score (0-1 scale)
resilience_df['vulnerability_score'] = resilience_df['yield_loss_pct']

# Categorize risk levels
resilience_df['risk_category'] = pd.cut(
    resilience_df['vulnerability_score'], 
    bins=[0, 0.20, 0.40, 1.0],
    labels=['Low Risk (<20% loss)', 'Medium Risk (20-40%)', 'High Risk (>40%)']
)

print("\nSourcing Distribution by Risk Level (2050, Current Policies):")
print("-" * 70)

risk_summary = resilience_df.groupby('risk_category').agg({
    'sourcing_pct': 'sum',
    'annual_mt': 'sum',
    'volume_at_risk_mt': 'sum',
    'profit_at_risk_M': 'sum'

}).reset_index()

for _, row in risk_summary.iterrows():
    print(f"\n{row['risk_category']}:")
    print(f"  Sourcing: {row['sourcing_pct']:.1f}% ({row['annual_mt']:,.0f} MT)")
    print(f"  Volume at risk: {row['volume_at_risk_mt']:,.0f} MT")
    print(f"  Profit at risk: ${row['profit_at_risk_M']:.1f}M")

# Diversification strategy
print("\n" + "="*70)
print("DIVERSIFICATION STRATEGY RECOMMENDATIONS")
print("="*70)

high_risk = resilience_df[resilience_df['risk_category'] == 'High Risk (>40%)'].sort_values(
    'sourcing_pct', ascending=False
)
low_risk = resilience_df[resilience_df['risk_category'] == 'Low Risk (<20% loss)'].sort_values(
    'annual_mt', ascending=False
)

print("\nHIGH-PRIORITY ACTIONS (High-risk regions to phase out):")
print("-" * 70)

if len(high_risk) > 0:
    for idx, row in high_risk.head(5).iterrows():
        replacement_needed = row['annual_mt']
        print(f"\n• {row['region_name']}, {row['country']}")
        print(f"  Current: {row['sourcing_pct']:.1f}% of supply ({row['annual_mt']:,.0f} MT)")
        print(f"  Risk: {row['yield_loss_pct']*100:.1f}% yield loss expected")
        print(f"  Action: Reduce sourcing by {replacement_needed * 0.5:,.0f} MT over 5 years")
        print(f"  Timeline: Begin transition by 2026")
else:
    print("  ✓ No regions in critical risk category")

print("\n\nEXPANSION OPPORTUNITIES (Low-risk regions to scale up):")
print("-" * 70)

if len(low_risk) > 0:
    for idx, row in low_risk.head(5).iterrows():
        expansion_potential = row['annual_mt'] * 0.5  # Could expand 50%
        print(f"\n• {row['region_name']}, {row['country']}")
        print(f"  Current: {row['sourcing_pct']:.1f}% of supply ({row['annual_mt']:,.0f} MT)")
        print(f"  Risk: Only {row['yield_loss_pct']*100:.1f}% yield loss expected")
        print(f"  Opportunity: Expand by {expansion_potential:,.0f} MT")
        print(f"  Investment needed: ~$" + f"{expansion_potential * 2000 / 1e6:.1f}M for infrastructure")
else:
    print("  ⚠ No low-risk regions available for expansion")

# Calculate optimal portfolio rebalancing
print("\n\nOPTIMAL PORTFOLIO REBALANCING:")
print("-" * 70)

total_volume = CONFIG['financial_params']['starbucks_annual_coffee_mt']
high_risk_volume = high_risk['annual_mt'].sum()
low_risk_volume = low_risk['annual_mt'].sum()

print(f"Current allocation:")
print(f"  High risk: {high_risk['sourcing_pct'].sum():.1f}% ({high_risk_volume:,.0f} MT)")
print(f"  Medium risk: {risk_summary[risk_summary['risk_category']=='Medium Risk (20-40%)']['sourcing_pct'].values[0]:.1f}%")
print(f"  Low risk: {low_risk['sourcing_pct'].sum():.1f}% ({low_risk_volume:,.0f} MT)")

print(f"\nRecommended target (by 2035):")
print(f"  High risk: <15% (reduce by {high_risk['sourcing_pct'].sum() - 15:.1f} percentage points)")
print(f"  Medium risk: 35-45%")
print(f"  Low risk: >40% (increase by {40 - low_risk['sourcing_pct'].sum():.1f} percentage points)")

# Save results
resilience_df.to_csv(f"{CONFIG['base_path']}/outputs/11_supply_chain_resilience.csv", index=False)
risk_summary.to_csv(f"{CONFIG['base_path']}/outputs/11_risk_distribution_summary.csv", index=False)

print("\n✓ Saved resilience and diversification analysis to outputs/")


STEP 11: SUPPLY CHAIN RESILIENCE & DIVERSIFICATION

Sourcing Distribution by Risk Level (2050, Current Policies):
----------------------------------------------------------------------

Low Risk (<20% loss):
  Sourcing: 84.8% (145,560 MT)
  Volume at risk: 6,879 MT
  Profit at risk: $141.9M

Medium Risk (20-40%):
  Sourcing: 4.0% (6,900 MT)
  Volume at risk: 1,625 MT
  Profit at risk: $33.5M

High Risk (>40%):
  Sourcing: 0.0% (0 MT)
  Volume at risk: 0 MT
  Profit at risk: $0.0M

DIVERSIFICATION STRATEGY RECOMMENDATIONS

HIGH-PRIORITY ACTIONS (High-risk regions to phase out):
----------------------------------------------------------------------
  ✓ No regions in critical risk category


EXPANSION OPPORTUNITIES (Low-risk regions to scale up):
----------------------------------------------------------------------

• Minas Gerais Sul, Brazil
  Current: 8.0% of supply (13,700 MT)
  Risk: Only 5.3% yield loss expected
  Opportunity: Expand by 6,850 MT
  Investment needed: ~$13.7M for inf

In [53]:
print("\n" + "="*80)
print("TOTAL CLIMATE RISK - 30-YEAR NET PRESENT VALUE")
print("="*80)

discount_rate = CONFIG['financial_params']['discount_rate']

total_npv_results = []

for scenario_code in ['ssp126', 'ssp245', 'ssp585']:
    scenario_name = scenario_map[scenario_code]
    
    mid_term = combined_df[
        (combined_df['scenario_code'] == scenario_code) &
        (combined_df['year'] == 2050)
    ]
    
    if not mid_term.empty:
        annual_cost = mid_term.iloc[0]['total_climate_cost_M']
        npv = sum([annual_cost / ((1 + discount_rate) ** t) for t in range(1, 31)])
        
        total_npv_results.append({
            'scenario': scenario_code,
            'scenario_name': scenario_name,
            'annual_cost_2050_M': annual_cost,
            'npv_30yr_M': npv,
            'pct_of_total_revenue': (npv / CONFIG['financial_params']['total_revenue']) * 100,
            'pct_of_total_assets': (npv / CONFIG['financial_params']['total_assets']) * 100
        })

total_npv_df = pd.DataFrame(total_npv_results)

print(f"\n{'Scenario':<30s} {'Annual 2050':<15s} {'30-Yr NPV':<15s} {'% Assets':<12s}")
print("-"*72)

for _, row in total_npv_df.iterrows():
    print(f"{row['scenario_name']:<30s} ${row['annual_cost_2050_M']:>12.1f}M ${row['npv_30yr_M']:>12.1f}M {row['pct_of_total_assets']:>10.2f}%")

print("="*80)

total_npv_df.to_csv(f"{CONFIG['base_path']}/outputs/total_climate_npv.csv", index=False)
print("\n✓ Saved to outputs/total_climate_npv.csv")


TOTAL CLIMATE RISK - 30-YEAR NET PRESENT VALUE

Scenario                       Annual 2050     30-Yr NPV       % Assets    
------------------------------------------------------------------------
Net Zero 2050                  $       995.7M $     11209.9M       0.00%
Delayed Transition             $      2453.8M $     27624.7M       0.00%
Current Policies               $       207.8M $      2339.2M       0.00%

✓ Saved to outputs/total_climate_npv.csv


In [54]:
# ============================================================================
# STEP 12: STRATEGIC PRICING RESPONSE (SIMPLIFIED)
# ============================================================================

print("\n" + "="*80)
print("STEP 12: STRATEGIC PRICING ANALYSIS")
print("="*80)

# Focus on most material scenario: Delayed Transition 2050
ssp245_2050 = combined_df[
    (combined_df['scenario_code'] == 'ssp245') & 
    (combined_df['year'] == 2050)
].iloc[0]

total_cost_M = ssp245_2050['total_climate_cost_M']
physical_M = ssp245_2050['physical_risk_M']
carbon_M = ssp245_2050['carbon_cost_M']

# Calculate per-kg impact
total_volume_kg = CONFIG['financial_params']['starbucks_annual_coffee_mt'] * 1000
cost_per_kg = (total_cost_M * 1_000_000) / total_volume_kg
current_revenue_per_kg = CONFIG['financial_params']['coffee_revenue'] / total_volume_kg

print(f"\nDelayed Transition Scenario (2050):")
print(f"  Total climate cost: ${total_cost_M:.1f}M")
print(f"    Physical: ${physical_M:.1f}M")
print(f"    Carbon: ${carbon_M:.1f}M")
print(f"\n  Cost per kg: ${cost_per_kg:.2f}")
print(f"  As % of current price: {cost_per_kg/current_revenue_per_kg*100:.1f}%")

print(f"\nPricing Strategy Options:")
print(f"  Option A - Full pass-through: +{cost_per_kg/current_revenue_per_kg*100:.1f}% price increase")
print(f"  Option B - Absorb 50%: +{cost_per_kg/current_revenue_per_kg*50:.1f}% price increase")
print(f"  Option C - Full absorption: Margin compression of {cost_per_kg/current_revenue_per_kg*100:.1f}%")

print(f"\nRecommendation: Gradual price increases starting 2030 to maintain margins")
print("="*80)


STEP 12: STRATEGIC PRICING ANALYSIS

Delayed Transition Scenario (2050):
  Total climate cost: $2453.8M
    Physical: $158.8M
    Carbon: $2295.0M

  Cost per kg: $14.27
  As % of current price: 19.4%

Pricing Strategy Options:
  Option A - Full pass-through: +19.4% price increase
  Option B - Absorb 50%: +9.7% price increase
  Option C - Full absorption: Margin compression of 19.4%

Recommendation: Gradual price increases starting 2030 to maintain margins


In [55]:
# ============================================================================
# STEP 14: COMPETITOR BENCHMARKING & MARKET POSITIONING  (FULLY FIXED)
# ============================================================================

print("\n" + "="*80)
print("STEP 14: COMPETITOR BENCHMARKING & MARKET POSITIONING")
print("="*80)

# ============================================================================
# COMPETITOR PROFILES (Based on public data & industry reports)
# ============================================================================

competitors = {
    'Starbucks': {
        'annual_volume_mt': CONFIG['financial_params']['starbucks_annual_coffee_mt'],
        'sourcing_regions': len(regions_df),
        'top3_concentration': None,   # Will be replaced with real calculated value
        'climate_disclosure': 'TCFD-aligned',
        'adaptation_investment_M': 0, # Will be updated from Step 13 if desired
        'sustainability_commitment': 'Net-zero by 2050'
    },
    'Nestlé (Nescafé)': {
        'annual_volume_mt': 230000,
        'sourcing_regions': 50,
        'top3_concentration': 0.40,
        'climate_disclosure': 'TCFD-compliant',
        'adaptation_investment_M': 300,
        'sustainability_commitment': 'Net-zero by 2050'
    },
    'JDE Peets': {
        'annual_volume_mt': 180000,
        'sourcing_regions': 35,
        'top3_concentration': 0.38,
        'climate_disclosure': 'Partial TCFD',
        'adaptation_investment_M': 150,
        'sustainability_commitment': 'Carbon neutral by 2040'
    },
    'Lavazza': {
        'annual_volume_mt': 60000,
        'sourcing_regions': 25,
        'top3_concentration': 0.45,
        'climate_disclosure': 'Basic ESG',
        'adaptation_investment_M': 80,
        'sustainability_commitment': 'Sustainable sourcing 100% by 2030'
    },
    'illycaffè': {
        'annual_volume_mt': 8000,
        'sourcing_regions': 12,
        'top3_concentration': 0.60,
        'climate_disclosure': 'Sustainability report',
        'adaptation_investment_M': 20,
        'sustainability_commitment': 'Carbon neutral by 2033'
    }
}

# ============================================================================
# CALCULATE STARBUCKS TOP-3 CONCENTRATION FROM YOUR DATA
# ============================================================================

regional_risk_585_2050 = yield_df[
    (yield_df['scenario'] == 'ssp585') &
    (yield_df['period'] == 'mid_term')
].sort_values('profit_at_risk_M', ascending=False)

starbucks_top3_concentration = regional_risk_585_2050.head(3)['sourcing_pct'].sum() / 100
competitors['Starbucks']['top3_concentration'] = starbucks_top3_concentration

print(f"Starbucks Top-3 Concentration (from YOUR analysis): {starbucks_top3_concentration:.1%}\n")

# ============================================================================
# GET STARBUCKS BASELINE CLIMATE COST (FROM Step 9 COMBINED_DF)
# ============================================================================

sbux_cost_2050 = combined_df[
    (combined_df['scenario_code'] == 'ssp585') &
    (combined_df['year'] == 2050)
]['total_climate_cost_M'].values[0]

# ============================================================================
# BUILD COMPETITOR COMPARISON TABLE
# ============================================================================

competitor_analysis = []

# Starbucks' own diversification and preparedness for scaling comparisons
sbux_regions = competitors['Starbucks']['sourcing_regions']
sbux_top3 = competitors['Starbucks']['top3_concentration']

# Diversification score for Starbucks
sbux_region_div_score = min(sbux_regions / 50, 1.0)
sbux_concentration_score = 1 - sbux_top3
sbux_diversification = (sbux_region_div_score * 0.5) + (sbux_concentration_score * 0.5)

sbux_preparedness = 1.0  # TCFD-aligned baseline
sbux_vulnerability = 1 - ((sbux_diversification * 0.6) + (sbux_preparedness * 0.4))

for company, profile in competitors.items():

    # ---------- Diversification Score ----------
    region_div = min(profile['sourcing_regions'] / 50, 1.0)
    conc_score = 1 - profile['top3_concentration']
    diversification_score = (region_div * 0.5) + (conc_score * 0.5)

    # ---------- Preparedness Score ----------
    disclosure_map = {
        'TCFD-aligned': 1.0,
        'TCFD-compliant': 0.9,
        'Partial TCFD': 0.6,
        'Sustainability report': 0.4,
        'Basic ESG': 0.3
    }
    disclosure_score = disclosure_map.get(profile['climate_disclosure'], 0.2)

    # Normalize investment
    if profile['annual_volume_mt'] > 0:
        invest_per_mt = profile['adaptation_investment_M'] / profile['annual_volume_mt']
        investment_score = min(invest_per_mt / 2, 1.0)
    else:
        investment_score = 0

    preparedness_score = (investment_score * 0.7) + (disclosure_score * 0.3)

    # ---------- Vulnerability ----------
    vulnerability_score = 1 - ((diversification_score * 0.6) + (preparedness_score * 0.4))

    # ---------- Climate Cost ----------
    if company == 'Starbucks':
        estimated_cost = sbux_cost_2050
    else:
        volume_ratio = profile['annual_volume_mt'] / competitors['Starbucks']['annual_volume_mt']
        vulnerability_adjustment = vulnerability_score / sbux_vulnerability
        estimated_cost = sbux_cost_2050 * volume_ratio * vulnerability_adjustment

    # Revenue baseline: assume $45/kg like Starbucks
    annual_revenue = profile['annual_volume_mt'] * 1000 * 45
    cost_pct_rev = (estimated_cost * 1e6 / annual_revenue) * 100 if annual_revenue > 0 else 0

    competitor_analysis.append({
        'company': company,
        'annual_volume_mt': profile['annual_volume_mt'],
        'sourcing_regions': profile['sourcing_regions'],
        'top3_concentration': profile['top3_concentration'],
        'diversification_score': diversification_score,
        'preparedness_score': preparedness_score,
        'vulnerability_score': vulnerability_score,
        'estimated_climate_cost_2050_M': estimated_cost,
        'cost_pct_revenue': cost_pct_rev,
        'climate_disclosure': profile['climate_disclosure'],
        'sustainability_commitment': profile['sustainability_commitment']
    })

competitor_df = pd.DataFrame(competitor_analysis)
competitor_df = competitor_df.sort_values('vulnerability_score', ascending=False)

# ============================================================================
# DISPLAY RESULTS
# ============================================================================

print("\nVULNERABILITY RANKING (Most → Least Vulnerable):")
print("-" * 70)

for idx, row in competitor_df.iterrows():
    
    vuln_color = "🔴" if row['vulnerability_score'] > 0.65 else "🟡" if row['vulnerability_score'] > 0.45 else "🟢"
    
    print(f"\n{vuln_color} {row['company']} — Vulnerability: {row['vulnerability_score']:.2f}")
    print(f"   Volume: {row['annual_volume_mt']:,.0f} MT")
    print(f"   Regions: {row['sourcing_regions']}")
    print(f"   Top-3 concentration: {row['top3_concentration']:.1%}")
    print(f"   Diversification score: {row['diversification_score']:.2f}")
    print(f"   Preparedness score: {row['preparedness_score']:.2f}")
    print(f"   Estimated 2050 climate cost: ${row['estimated_climate_cost_2050_M']:.0f}M")
    print(f"   Cost as % of revenue: {row['cost_pct_revenue']:.1f}%")
    print(f"   Disclosure: {row['climate_disclosure']}")

# ============================================================================
# STARBUCKS COMPETITIVE POSITIONING
# ============================================================================

print("\n" + "="*70)
print("STARBUCKS COMPETITIVE POSITION")
print("="*70)

sbux_row = competitor_df[competitor_df['company'] == 'Starbucks'].iloc[0]
sbux_rank = competitor_df.index.get_loc(sbux_row.name) + 1

print(f"\nStarbucks ranks: #{sbux_rank} (1 = most vulnerable)")
print(f"  Diversification: {sbux_row['diversification_score']:.2f}")
print(f"  Preparedness: {sbux_row['preparedness_score']:.2f}")
print(f"  Vulnerability: {sbux_row['vulnerability_score']:.2f}")

print("\nPotential improvement with adaptation:")
print(f"  Diversification: {sbux_row['diversification_score']:.2f} → {min(sbux_row['diversification_score']+0.15,1):.2f}")
print(f"  Preparedness: {sbux_row['preparedness_score']:.2f} → {min(sbux_row['preparedness_score']+0.25,1):.2f}")

# ============================================================================
# SAVE
# ============================================================================

competitor_df.to_csv(f"{CONFIG['base_path']}/outputs/14_competitor_benchmarking.csv", index=False)

print("\n✓ Saved competitor benchmarking analysis.\n")



STEP 14: COMPETITOR BENCHMARKING & MARKET POSITIONING
Starbucks Top-3 Concentration (from YOUR analysis): 17.5%


VULNERABILITY RANKING (Most → Least Vulnerable):
----------------------------------------------------------------------

🔴 illycaffè — Vulnerability: 0.76
   Volume: 8,000 MT
   Regions: 12
   Top-3 concentration: 60.0%
   Diversification score: 0.32
   Preparedness score: 0.12
   Estimated 2050 climate cost: $41M
   Cost as % of revenue: 11.4%
   Disclosure: Sustainability report

🟡 Lavazza — Vulnerability: 0.65
   Volume: 60,000 MT
   Regions: 25
   Top-3 concentration: 45.0%
   Diversification score: 0.53
   Preparedness score: 0.09
   Estimated 2050 climate cost: $263M
   Cost as % of revenue: 9.8%
   Disclosure: Basic ESG

🟡 JDE Peets — Vulnerability: 0.53
   Volume: 180,000 MT
   Regions: 35
   Top-3 concentration: 38.0%
   Diversification score: 0.66
   Preparedness score: 0.18
   Estimated 2050 climate cost: $648M
   Cost as % of revenue: 8.0%
   Disclosure: Partia

In [56]:
# ============================================================================
# STEP 15: CARBON PRICE RISK HEDGING
# ============================================================================

print("\n" + "="*80)
print("STEP 15: CARBON PRICE HEDGING STRATEGY")
print("="*80)

# Focus on Delayed Transition scenario ($2,295M carbon cost)
baseline_carbon_cost_M = 2295  # From Step 9

print(f"BASELINE CARBON EXPOSURE (Delayed Transition, 2050):")
print(f"  Annual carbon cost: ${baseline_carbon_cost_M:.0f}M")
print(f"  Price per ton: $306/tCO2")
print()

# Carbon hedging strategies
hedging_strategies = [
    {'strategy': 'No Hedge', 'hedge_ratio': 0.0, 'cost_pct': 0},
    {'strategy': '25% Carbon Swaps', 'hedge_ratio': 0.25, 'cost_pct': 3},
    {'strategy': '50% Carbon Futures', 'hedge_ratio': 0.50, 'cost_pct': 5},
    {'strategy': '75% Collar Strategy', 'hedge_ratio': 0.75, 'cost_pct': 7},
]

for strategy in hedging_strategies:
    hedged_amount_M = baseline_carbon_cost_M * strategy['hedge_ratio']
    hedge_cost_M = hedged_amount_M * strategy['cost_pct'] / 100
    
    # If carbon price spikes to $500/ton (from $306)
    price_spike_pct = 0.63  # (500-306)/306
    savings_if_spike_M = hedged_amount_M * price_spike_pct
    net_benefit_M = savings_if_spike_M - hedge_cost_M
    
    print(f"\n{strategy['strategy']}:")
    print(f"  Hedged exposure: ${hedged_amount_M:.0f}M")
    print(f"  Annual cost: ${hedge_cost_M:.1f}M")
    print(f"  Savings if spike: ${savings_if_spike_M:.0f}M")
    print(f"  Net benefit: ${net_benefit_M:.0f}M")

print("\nRecommendation: 25-50% carbon price hedge to manage transition risk volatility")


STEP 15: CARBON PRICE HEDGING STRATEGY
BASELINE CARBON EXPOSURE (Delayed Transition, 2050):
  Annual carbon cost: $2295M
  Price per ton: $306/tCO2


No Hedge:
  Hedged exposure: $0M
  Annual cost: $0.0M
  Savings if spike: $0M
  Net benefit: $0M

25% Carbon Swaps:
  Hedged exposure: $574M
  Annual cost: $17.2M
  Savings if spike: $361M
  Net benefit: $344M

50% Carbon Futures:
  Hedged exposure: $1148M
  Annual cost: $57.4M
  Savings if spike: $723M
  Net benefit: $666M

75% Collar Strategy:
  Hedged exposure: $1721M
  Annual cost: $120.5M
  Savings if spike: $1084M
  Net benefit: $964M

Recommendation: 25-50% carbon price hedge to manage transition risk volatility


new visuals

In [57]:
# ============================================================================
# STEP 17A: EMISSION TRAJECTORY (ORIGINAL OUTPUT)
# ============================================================================

print("="*70)
print("STEP 17A: NET-ZERO EMISSION TRAJECTORY (ORIGINAL FORMAT)")
print("="*70)

# Baseline Starbucks emissions (2023 ESG Report)
scope_1_baseline = 0.3
scope_2_baseline = 1.2
scope_3_baseline = 13.8
total_baseline = scope_1_baseline + scope_2_baseline + scope_3_baseline

print(f"\nACTUAL STARBUCKS BASELINE EMISSIONS (2023):")
print(f"  Scope 1: {scope_1_baseline:.1f} M MT CO2e")
print(f"  Scope 2: {scope_2_baseline:.1f} M MT CO2e")
print(f"  Scope 3: {scope_3_baseline:.1f} M MT CO2e")
print(f"  TOTAL BASELINE: {total_baseline:.1f} M MT CO2e")

# Starbucks official targets
target_2030_reduction = 0.50     # 50% by 2030
target_2050_reduction = 0.95     # 95% by 2050 (net-zero with 5% offsets)

target_2030 = total_baseline * (1 - target_2030_reduction)
target_2050 = total_baseline * (1 - target_2050_reduction)

print(f"\nOFFICIAL STARBUCKS NET-ZERO TARGETS:")
print(f"  2030: {target_2030:.2f} M MT CO2e (50% cut)")
print(f"  2050: {target_2050:.2f} M MT CO2e (95% cut)")

# Emission trajectory: BAU vs Net-Zero
years = [2024, 2030, 2035, 2040, 2045, 2050]
growth_rate = 1.02  # BAU = 2%/yr growth

business_as_usual = [
    total_baseline * (growth_rate ** (year - 2024)) 
    for year in years
]

net_zero_trajectory = [
    total_baseline,         # 2024
    target_2030,            # 2030: 50% reduction
    total_baseline * 0.42,  # interpolated to 2035
    total_baseline * 0.18,  # 2040
    total_baseline * 0.10,  # 2045
    target_2050             # 2050
]

print("\nYear     Business-as-Usual    Net-Zero Pathway     Reduction    Reduction %")
print("-" * 75)
for i, year in enumerate(years):
    bau = business_as_usual[i]
    nz = net_zero_trajectory[i]
    reduction = bau - nz
    reduction_pct = (reduction / bau) * 100 if bau > 0 else 0
    print(f"{year:<8} {bau:<20.2f} {nz:<20.2f} {reduction:<12.2f} {reduction_pct:>9.1f}%")

# Save emissions trajectory
trajectory_df = pd.DataFrame({
    'year': years,
    'business_as_usual_MT': business_as_usual,
    'net_zero_MT': net_zero_trajectory,
})
trajectory_df.to_csv(f"{CONFIG['base_path']}/outputs/17a_emission_trajectory.csv", index=False)

print("\n✓ Saved original emission trajectory to outputs/17a_emission_trajectory.csv")


STEP 17A: NET-ZERO EMISSION TRAJECTORY (ORIGINAL FORMAT)

ACTUAL STARBUCKS BASELINE EMISSIONS (2023):
  Scope 1: 0.3 M MT CO2e
  Scope 2: 1.2 M MT CO2e
  Scope 3: 13.8 M MT CO2e
  TOTAL BASELINE: 15.3 M MT CO2e

OFFICIAL STARBUCKS NET-ZERO TARGETS:
  2030: 7.65 M MT CO2e (50% cut)
  2050: 0.77 M MT CO2e (95% cut)

Year     Business-as-Usual    Net-Zero Pathway     Reduction    Reduction %
---------------------------------------------------------------------------
2024     15.30                15.30                0.00               0.0%
2030     17.23                7.65                 9.58              55.6%
2035     19.02                6.43                 12.60             66.2%
2040     21.00                2.75                 18.25             86.9%
2045     23.19                1.53                 21.66             93.4%
2050     25.60                0.77                 24.84             97.0%

✓ Saved original emission trajectory to outputs/17a_emission_trajectory.csv


In [58]:
# ============================================================================
# STEP 17B: NET-ZERO FINANCIAL IMPACT (PIPELINE-INTEGRATED)
# ============================================================================

print("\n" + "="*70)
print("STEP 17B: NET-ZERO FINANCIAL IMPACT USING REAL PHYSICAL + TRANSITION RISKS")
print("="*70)

# Pull climate risk values from your model (SSP585 worst case)
phys_2030 = combined_df[(combined_df['scenario_code']=='ssp585') & (combined_df['year']==2030)]
phys_2050 = combined_df[(combined_df['scenario_code']=='ssp585') & (combined_df['year']==2050)]

physical_2030_M = float(phys_2030['physical_risk_M'])
physical_2050_M = float(phys_2050['physical_risk_M'])
carbon_2030_M   = float(phys_2030['carbon_cost_M'])
carbon_2050_M   = float(phys_2050['carbon_cost_M'])
total_2030_M    = float(phys_2030['total_climate_cost_M'])
total_2050_M    = float(phys_2050['total_climate_cost_M'])

print("\nREAL CLIMATE RISK METRICS (SSP585):")
print(f"  2030 total climate cost: ${total_2030_M:,.1f}M")
print(f"     - Physical: ${physical_2030_M:,.1f}M")
print(f"     - Carbon:   ${carbon_2030_M:,.1f}M")
print(f"  2050 total climate cost: ${total_2050_M:,.1f}M")
print(f"     - Physical: ${physical_2050_M:,.1f}M")
print(f"     - Carbon:   ${carbon_2050_M:,.1f}M")

# Cost of inaction = total climate cost
cost_of_inaction_2030 = total_2030_M
cost_of_inaction_2050 = total_2050_M

# Cost of net-zero (decarbonization investment + reduced risk exposure)
total_investment = 1700  # $M through 2050

carbon_cost_2030_nz = target_2030 * 75
carbon_cost_2050_nz = target_2050 * 150

supply_2030_nz = physical_2030_M * 0.35
supply_2050_nz = physical_2050_M * 0.10

cost_of_nz_2030 = carbon_cost_2030_nz + supply_2030_nz + (total_investment * 0.15)
cost_of_nz_2050 = carbon_cost_2050_nz + supply_2050_nz + (total_investment * 0.10)

avoided_cost_2030 = cost_of_inaction_2030 - cost_of_nz_2030
avoided_cost_2050 = cost_of_inaction_2050 - cost_of_nz_2050

print("\nFINANCIAL IMPACT:")
print(f"  Avoided cost (2030): ${avoided_cost_2030:,.0f}M")
print(f"  Avoided cost (2050): ${avoided_cost_2050:,.0f}M")

avg_annual_benefit = (avoided_cost_2030 + avoided_cost_2050) / 2
net_benefit = avg_annual_benefit * 26 - total_investment

print("\nNET FINANCIAL BENEFIT OF NET-ZERO:")
print(f"  30-year avoided climate cost: ${avg_annual_benefit*26:,.0f}M")
print(f"  Net benefit after investment: ${net_benefit:,.0f}M")
print(f"  ROI: {net_benefit / total_investment * 100:.0f}%")

# Save results
nz_df = pd.DataFrame({
    'year': [2030, 2050],
    'cost_of_inaction_M': [cost_of_inaction_2030, cost_of_inaction_2050],
    'cost_of_net_zero_M': [cost_of_nz_2030, cost_of_nz_2050],
    'avoided_cost_M': [avoided_cost_2030, avoided_cost_2050],
})
nz_df.to_csv(f"{CONFIG['base_path']}/outputs/17b_net_zero_financial_analysis.csv", index=False)

print("\n✓ Saved net-zero financial analysis to outputs/17b_net_zero_financial_analysis.csv")



STEP 17B: NET-ZERO FINANCIAL IMPACT USING REAL PHYSICAL + TRANSITION RISKS

REAL CLIMATE RISK METRICS (SSP585):
  2030 total climate cost: $158.5M
     - Physical: $147.9M
     - Carbon:   $10.6M
  2050 total climate cost: $207.8M
     - Physical: $175.4M
     - Carbon:   $32.4M

FINANCIAL IMPACT:
  Avoided cost (2030): $-722M
  Avoided cost (2050): $-95M

NET FINANCIAL BENEFIT OF NET-ZERO:
  30-year avoided climate cost: $-10,615M
  Net benefit after investment: $-12,315M
  ROI: -724%

✓ Saved net-zero financial analysis to outputs/17b_net_zero_financial_analysis.csv


In [59]:
# ============================================================================
# VIZ 1: EXECUTIVE DASHBOARD - COMPLETE STORY
# ============================================================================
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import numpy as np
import pandas as pd

print("\n" + "="*80)
print("CREATING PORTFOLIO VISUALIZATIONS")
print("="*80)

# Setup
output_dir = f"{CONFIG['base_path']}/outputs/visualizations"
import os
os.makedirs(output_dir, exist_ok=True)

sns.set_style("whitegrid")
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'sans-serif'

print("\n1. Creating Executive Dashboard...")

fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.35, wspace=0.3)

fig.suptitle('STARBUCKS CLIMATE RISK ASSESSMENT - EXECUTIVE DASHBOARD\nTCFD-Aligned Analysis (2025-2050)', 
             fontsize=18, fontweight='bold', y=0.98)

# ============================================================================
# PANEL 1 (Top): TOTAL CLIMATE COST BY SCENARIO - 2050
# ============================================================================
ax1 = fig.add_subplot(gs[0, :])

scenarios_2050 = combined_df[combined_df['year'] == 2050].copy()
scenario_order = ['ssp126', 'ssp245', 'ssp585']
scenario_names = ['Net-Zero 2050', 'Delayed Transition', 'Current Policies']

totals = [scenarios_2050[scenarios_2050['scenario_code'] == s]['total_climate_cost_M'].values[0] 
          for s in scenario_order]
physical = [scenarios_2050[scenarios_2050['scenario_code'] == s]['physical_risk_M'].values[0] 
            for s in scenario_order]
carbon = [scenarios_2050[scenarios_2050['scenario_code'] == s]['carbon_cost_M'].values[0] 
          for s in scenario_order]

colors = ['#2ecc71', '#f39c12', '#e74c3c']
bars = ax1.bar(scenario_names, totals, color=colors, edgecolor='black', linewidth=2, alpha=0.85)

for bar, total, phys, carb in zip(bars, totals, physical, carbon):
    height = bar.get_height()
    # Total on top
    ax1.text(bar.get_x() + bar.get_width()/2., height + max(totals)*0.02,
            f'${total:,.0f}M', ha='center', va='bottom', fontweight='bold', fontsize=12)
    
    # Components inside
    ax1.text(bar.get_x() + bar.get_width()/2., height*0.5,
            f'Physical: ${phys:.0f}M\nCarbon: ${carb:.0f}M', 
            ha='center', va='center', fontsize=9, color='white', fontweight='bold')

ax1.set_ylabel('Total Annual Climate Cost ($M)', fontsize=12, fontweight='bold')
ax1.set_title('Total Climate Risk by Scenario (2050)', fontsize=14, fontweight='bold', pad=15)
ax1.yaxis.grid(True, alpha=0.3)
ax1.set_ylim(0, max(totals) * 1.15)

# Add key insight
insight_text = f"Transition risks (carbon) are {carbon[1]/physical[1]:.0f}x larger than physical risks"
ax1.text(0.98, 0.95, insight_text, transform=ax1.transAxes,
         bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7),
         fontsize=10, fontweight='bold', ha='right', va='top')

# ============================================================================
# PANEL 2 (Middle-Left): REGIONAL RISK DISTRIBUTION PIE
# ============================================================================
ax2 = fig.add_subplot(gs[1, 0])

regional_2050 = yield_df[
    (yield_df['scenario'] == 'ssp585') &
    (yield_df['period'] == 'mid_term')
].copy()

# Risk categories based on yield loss
low_risk = (regional_2050['yield_loss_pct'] < 0.20).sum()
med_risk = ((regional_2050['yield_loss_pct'] >= 0.20) & 
            (regional_2050['yield_loss_pct'] < 0.40)).sum()
high_risk = (regional_2050['yield_loss_pct'] >= 0.40).sum()

sizes = [low_risk, med_risk, high_risk]
labels = [f'Low Risk\n(<20% loss)\n{low_risk} regions', 
          f'Medium Risk\n(20-40% loss)\n{med_risk} regions',
          f'High Risk\n(>40% loss)\n{high_risk} regions']
colors_pie = ['#2ecc71', '#f39c12', '#e74c3c']

wedges, texts, autotexts = ax2.pie(sizes, labels=labels, autopct='%1.1f%%',
                                     colors=colors_pie, startangle=90,
                                     textprops={'fontsize': 9, 'fontweight': 'bold'})

for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontsize(11)

ax2.set_title('Regional Vulnerability Distribution\n(SSP5-8.5, 2050)', 
              fontsize=11, fontweight='bold', pad=10)

# ============================================================================
# PANEL 3 (Middle-Center): PHYSICAL VS TRANSITION RISK
# ============================================================================
ax3 = fig.add_subplot(gs[1, 1])

x = np.arange(len(scenario_names))
width = 0.35

bars1 = ax3.bar(x - width/2, physical, width, label='Physical Risk',
                color='#e74c3c', edgecolor='black', linewidth=1.5, alpha=0.85)
bars2 = ax3.bar(x + width/2, carbon, width, label='Transition Risk (Carbon)',
                color='#3498db', edgecolor='black', linewidth=1.5, alpha=0.85)

ax3.set_ylabel('Annual Cost ($M)', fontsize=10, fontweight='bold')
ax3.set_title('Physical vs Transition Risk\nBreakdown by Scenario', 
              fontsize=11, fontweight='bold', pad=10)
ax3.set_xticks(x)
ax3.set_xticklabels(scenario_names, fontsize=8, rotation=15, ha='right')
ax3.legend(fontsize=9, loc='upper left')
ax3.yaxis.grid(True, alpha=0.3)

# ============================================================================
# PANEL 4 (Middle-Right): KEY METRICS BOX
# ============================================================================
ax4 = fig.add_subplot(gs[1, 2])
ax4.axis('off')

# Calculate key metrics
most_likely_total = totals[1]  # SSP245
most_likely_physical = physical[1]
most_likely_carbon = carbon[1]
total_regions = len(regional_2050)
low_risk_pct = (low_risk / total_regions) * 100

metrics_text = f"""
KEY FINDINGS

📊 Total Exposure
   ${most_likely_total:,.0f}M
   (Delayed Transition, 2050)

🌱 Physical Risk
   ${most_likely_physical:.0f}M/year
   (Coffee supply impacts)

💨 Transition Risk
   ${most_likely_carbon:,.0f}M/year
   (Carbon pricing)

🗺️ Portfolio Resilience
   {low_risk_pct:.0f}% Low Risk
   ({low_risk}/{total_regions} regions)

⚠️ High-Risk Regions
   {high_risk} regions
   ({(high_risk/total_regions)*100:.0f}% of portfolio)
"""

ax4.text(0.05, 0.95, metrics_text, transform=ax4.transAxes,
         fontsize=10, fontweight='bold', verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.3),
         family='monospace')

# ============================================================================
# PANEL 5 (Bottom): TOP 10 VULNERABLE REGIONS BAR CHART
# ============================================================================
ax5 = fig.add_subplot(gs[2, :])

top10 = regional_2050.nlargest(10, 'yield_loss_pct')[['region_name', 'country', 'yield_loss_pct', 'sourcing_pct']].copy()
top10['label'] = top10['region_name'] + ', ' + top10['country']

y_pos = np.arange(len(top10))
colors_bar = ['#e74c3c' if x >= 0.40 else '#f39c12' if x >= 0.20 else '#2ecc71' 
              for x in top10['yield_loss_pct']]

bars = ax5.barh(y_pos, top10['yield_loss_pct'] * 100, color=colors_bar, 
                edgecolor='black', linewidth=1.5, alpha=0.85)

ax5.set_yticks(y_pos)
ax5.set_yticklabels(top10['label'], fontsize=9)
ax5.invert_yaxis()
ax5.set_xlabel('Expected Yield Loss (%)', fontsize=11, fontweight='bold')
ax5.set_title('Top 10 Most Vulnerable Regions (SSP5-8.5, 2050)', 
              fontsize=12, fontweight='bold', pad=15)
ax5.xaxis.grid(True, alpha=0.3)

# Add sourcing % labels
for i, (bar, loss, pct) in enumerate(zip(bars, top10['yield_loss_pct'], top10['sourcing_pct'])):
    ax5.text(loss * 100 + 0.5, i, f'{loss*100:.1f}% loss | {pct:.1f}% sourcing',
             va='center', fontsize=8, fontweight='bold')

plt.tight_layout()
plt.savefig(f"{output_dir}/01_Executive_Dashboard.png", dpi=300, bbox_inches='tight')
plt.close()
print("   ✓ Saved: 01_Executive_Dashboard.png")


CREATING PORTFOLIO VISUALIZATIONS

1. Creating Executive Dashboard...
   ✓ Saved: 01_Executive_Dashboard.png


In [60]:
print("\n2. Creating Scenario Comparison Timeline...")

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))
fig.suptitle('Climate Risk Evolution: 2030 vs 2050 Comparison', 
             fontsize=16, fontweight='bold')

# Get 2030 and 2050 data
years_data = combined_df[combined_df['year'].isin([2030, 2050])].copy()

for scenario_code, scenario_name in zip(['ssp126', 'ssp245', 'ssp585'], 
                                         ['Net-Zero 2050', 'Delayed Transition', 'Current Policies']):
    scenario_data = years_data[years_data['scenario_code'] == scenario_code]
    
    years = scenario_data['year'].values
    physical_vals = scenario_data['physical_risk_M'].values
    carbon_vals = scenario_data['carbon_cost_M'].values
    total_vals = scenario_data['total_climate_cost_M'].values
    
    # Plot 1: Total cost evolution
    ax1.plot(years, total_vals, marker='o', linewidth=3, markersize=10, 
             label=scenario_name, alpha=0.8)
    
    # Add value labels
    for year, val in zip(years, total_vals):
        ax1.text(year, val + max(total_vals)*0.02, f'${val:,.0f}M',
                ha='center', fontsize=9, fontweight='bold')

ax1.set_xlabel('Year', fontsize=12, fontweight='bold')
ax1.set_ylabel('Total Climate Cost ($M)', fontsize=12, fontweight='bold')
ax1.set_title('Total Climate Risk Trajectory by Scenario', fontsize=13, fontweight='bold')
ax1.legend(fontsize=11, loc='upper left')
ax1.grid(True, alpha=0.3)
ax1.set_xticks([2030, 2050])

# Plot 2: Physical vs Carbon breakdown for 2050
scenarios_2050_sorted = scenarios_2050.sort_values('scenario_code')
x = np.arange(len(scenario_names))
width = 0.35

bars1 = ax2.bar(x - width/2, physical, width, label='Physical Risk',
                color='#e74c3c', edgecolor='black', linewidth=1.5, alpha=0.85)
bars2 = ax2.bar(x + width/2, carbon, width, label='Carbon Cost',
                color='#3498db', edgecolor='black', linewidth=1.5, alpha=0.85)

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height + max(carbon)*0.02,
                f'${height:,.0f}M', ha='center', va='bottom', fontsize=10, fontweight='bold')

ax2.set_ylabel('Annual Cost ($M)', fontsize=12, fontweight='bold')
ax2.set_title('Physical vs Transition Risk Comparison (2050)', fontsize=13, fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels(scenario_names, fontsize=11)
ax2.legend(fontsize=11)
ax2.yaxis.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f"{output_dir}/02_Scenario_Timeline.png", dpi=300, bbox_inches='tight')
plt.close()
print("   ✓ Saved: 02_Scenario_Timeline.png")


2. Creating Scenario Comparison Timeline...
   ✓ Saved: 02_Scenario_Timeline.png


In [61]:
print("\n3. Creating Geographic Vulnerability Visualization...")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))
fig.suptitle('Geographic Climate Risk Assessment', fontsize=16, fontweight='bold')

# LEFT: Bubble chart - Stress vs Sourcing %
regional_sorted = regional_2050.sort_values('yield_loss_pct', ascending=False).head(20)

scatter = ax1.scatter(regional_sorted['sourcing_pct'], 
                      regional_sorted['yield_loss_pct'] * 100,
                      s=regional_sorted['profit_at_risk_M'] * 10,  # Size by financial impact
                      c=regional_sorted['stress_index'],
                      cmap='RdYlGn_r', alpha=0.7, edgecolors='black', linewidth=1.5)

# Add region labels for top 10
for idx, row in regional_sorted.head(10).iterrows():
    ax1.annotate(row['region_name'], 
                (row['sourcing_pct'], row['yield_loss_pct'] * 100),
                xytext=(5, 5), textcoords='offset points',
                fontsize=8, fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.5))

ax1.set_xlabel('% of Total Sourcing', fontsize=11, fontweight='bold')
ax1.set_ylabel('Expected Yield Loss (%)', fontsize=11, fontweight='bold')
ax1.set_title('Climate Risk vs Portfolio Concentration\n(Bubble size = Profit at risk)', 
              fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Add risk zones
ax1.axhline(y=20, color='orange', linestyle='--', linewidth=2, alpha=0.5, label='Medium risk threshold')
ax1.axhline(y=40, color='red', linestyle='--', linewidth=2, alpha=0.5, label='High risk threshold')
ax1.legend(fontsize=9)

cbar = plt.colorbar(scatter, ax=ax1)
cbar.set_label('Climate Stress Index', fontweight='bold')

# RIGHT: Regional concentration bar chart
top_regions = regional_2050.nlargest(15, 'sourcing_pct')[['region_name', 'country', 'sourcing_pct', 'yield_loss_pct']].copy()
top_regions['label'] = top_regions['region_name'] + ', ' + top_regions['country']

y_pos = np.arange(len(top_regions))
colors_conc = ['#e74c3c' if x >= 0.40 else '#f39c12' if x >= 0.20 else '#2ecc71' 
               for x in top_regions['yield_loss_pct']]

bars = ax2.barh(y_pos, top_regions['sourcing_pct'], color=colors_conc,
                edgecolor='black', linewidth=1.5, alpha=0.85)

ax2.set_yticks(y_pos)
ax2.set_yticklabels(top_regions['label'], fontsize=9)
ax2.invert_yaxis()
ax2.set_xlabel('% of Total Sourcing', fontsize=11, fontweight='bold')
ax2.set_title('Top 15 Sourcing Regions\n(Color = Risk Level)', fontsize=12, fontweight='bold')
ax2.xaxis.grid(True, alpha=0.3)

# Add sourcing % labels
for i, (bar, pct, loss) in enumerate(zip(bars, top_regions['sourcing_pct'], top_regions['yield_loss_pct'])):
    ax2.text(pct + 0.2, i, f'{pct:.1f}%', va='center', fontsize=8, fontweight='bold')

plt.tight_layout()
plt.savefig(f"{output_dir}/03_Geographic_Vulnerability.png", dpi=300, bbox_inches='tight')
plt.close()
print("   ✓ Saved: 03_Geographic_Vulnerability.png")


3. Creating Geographic Vulnerability Visualization...
   ✓ Saved: 03_Geographic_Vulnerability.png


In [63]:
# ============================================================================
# VIZ 4: COMPETITIVE BENCHMARKING - INDUSTRY CLIMATE RESILIENCE
# ============================================================================

print("\n4. Creating Competitive Benchmarking Chart...")

# Check if competitor_df exists
if 'competitor_df' not in globals():
    print("   ⚠️ competitor_df not found - run Step 14 first")
else:
    fig = plt.figure(figsize=(16, 12))
    gs = fig.add_gridspec(3, 2, hspace=0.35, wspace=0.3)
    
    fig.suptitle('COMPETITIVE CLIMATE RISK BENCHMARKING\nIndustry Vulnerability Assessment', 
                 fontsize=18, fontweight='bold', y=0.98)
    
    # Sort by vulnerability (best to worst)
    comp_sorted = competitor_df.sort_values('vulnerability_score', ascending=True)
    companies = comp_sorted['company'].tolist()
    
    # ========================================================================
    # PANEL 1 (Top-Left): OVERALL VULNERABILITY RANKING
    # ========================================================================
    ax1 = fig.add_subplot(gs[0, :])
    
    vulnerability = comp_sorted['vulnerability_score'].values
    
    # Color code: green = best, red = worst
    colors_vuln = []
    for v in vulnerability:
        if v < 0.45:
            colors_vuln.append('#2ecc71')  # Green - low vulnerability
        elif v < 0.55:
            colors_vuln.append('#f39c12')  # Orange - medium
        else:
            colors_vuln.append('#e74c3c')  # Red - high vulnerability
    
    bars = ax1.barh(companies, vulnerability, color=colors_vuln, 
                    edgecolor='black', linewidth=2, alpha=0.85)
    
    # Highlight Starbucks
    starbucks_idx = companies.index('Starbucks')
    bars[starbucks_idx].set_edgecolor('gold')
    bars[starbucks_idx].set_linewidth(4)
    
    # Add value labels
    for i, (bar, vuln) in enumerate(zip(bars, vulnerability)):
        label_text = f'{vuln:.2f}'
        if companies[i] == 'Starbucks':
            label_text += ' ⭐'
        ax1.text(vuln + 0.02, bar.get_y() + bar.get_height()/2,
                label_text, va='center', fontsize=11, fontweight='bold')
    
    ax1.set_xlabel('Vulnerability Score (Lower = Better)', fontsize=12, fontweight='bold')
    ax1.set_title('Overall Climate Vulnerability Ranking', fontsize=14, fontweight='bold', pad=15)
    ax1.set_xlim(0, 1.0)
    ax1.xaxis.grid(True, alpha=0.3)
    ax1.invert_yaxis()
    
    # Add legend
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='#2ecc71', edgecolor='black', label='Low Vulnerability (<0.45)'),
        Patch(facecolor='#f39c12', edgecolor='black', label='Medium Vulnerability (0.45-0.55)'),
        Patch(facecolor='#e74c3c', edgecolor='black', label='High Vulnerability (>0.55)'),
        Patch(facecolor='white', edgecolor='gold', linewidth=3, label='Starbucks')
    ]
    ax1.legend(handles=legend_elements, loc='lower right', fontsize=10)
    
    # ========================================================================
    # PANEL 2 (Middle-Left): DIVERSIFICATION SCORES
    # ========================================================================
    ax2 = fig.add_subplot(gs[1, 0])
    
    diversification = comp_sorted['diversification_score'].values
    preparedness = comp_sorted['preparedness_score'].values
    
    x_pos = np.arange(len(companies))
    width = 0.35
    
    bars1 = ax2.barh(x_pos - width/2, diversification, width, 
                     label='Diversification', color='#3498db', 
                     edgecolor='black', linewidth=1.5, alpha=0.85)
    bars2 = ax2.barh(x_pos + width/2, preparedness, width,
                     label='Preparedness', color='#9b59b6',
                     edgecolor='black', linewidth=1.5, alpha=0.85)
    
    ax2.set_yticks(x_pos)
    ax2.set_yticklabels(companies, fontsize=10)
    ax2.invert_yaxis()
    ax2.set_xlabel('Score (0-1)', fontsize=11, fontweight='bold')
    ax2.set_title('Diversification & Preparedness', fontsize=12, fontweight='bold', pad=10)
    ax2.legend(fontsize=9, loc='lower right')
    ax2.xaxis.grid(True, alpha=0.3)
    ax2.set_xlim(0, 1.0)
    
    # ========================================================================
    # PANEL 3 (Middle-Right): TOP-3 CONCENTRATION RISK
    # ========================================================================
    ax3 = fig.add_subplot(gs[1, 1])
    
    concentration = comp_sorted['top3_concentration'].values * 100
    
    # Color by risk level
    colors_conc = ['#e74c3c' if c > 50 else '#f39c12' if c > 35 else '#2ecc71' 
                   for c in concentration]
    
    bars = ax3.barh(companies, concentration, color=colors_conc,
                    edgecolor='black', linewidth=1.5, alpha=0.85)
    
    # Highlight Starbucks
    bars[starbucks_idx].set_edgecolor('gold')
    bars[starbucks_idx].set_linewidth(3)
    
    # Add value labels
    for bar, conc in zip(bars, concentration):
        ax3.text(conc + 1, bar.get_y() + bar.get_height()/2,
                f'{conc:.1f}%', va='center', fontsize=10, fontweight='bold')
    
    ax3.invert_yaxis()
    ax3.set_xlabel('Top-3 Regional Concentration (%)', fontsize=11, fontweight='bold')
    ax3.set_title('Geographic Concentration Risk\n(Lower = Better Diversified)', 
                  fontsize=12, fontweight='bold', pad=10)
    ax3.xaxis.grid(True, alpha=0.3)
    
    # Add risk zones
    ax3.axvline(x=50, color='red', linestyle='--', linewidth=2, alpha=0.5)
    ax3.axvline(x=35, color='orange', linestyle='--', linewidth=2, alpha=0.5)
    ax3.text(50, len(companies)*0.9, 'High Risk', ha='center', fontsize=8, 
             bbox=dict(boxstyle='round', facecolor='red', alpha=0.3))
    ax3.text(35, len(companies)*0.9, 'Medium', ha='center', fontsize=8,
             bbox=dict(boxstyle='round', facecolor='orange', alpha=0.3))
    
    # ========================================================================
    # PANEL 4 (Bottom-Left): ESTIMATED 2050 CLIMATE COST
    # ========================================================================
    ax4 = fig.add_subplot(gs[2, 0])
    
    climate_costs = comp_sorted['estimated_climate_cost_2050_M'].values
    
    bars = ax4.barh(companies, climate_costs, color='#e74c3c',
                    edgecolor='black', linewidth=1.5, alpha=0.85)
    
    # Highlight Starbucks
    bars[starbucks_idx].set_color('#f39c12')
    bars[starbucks_idx].set_edgecolor('gold')
    bars[starbucks_idx].set_linewidth(3)
    
    # Add value labels
    for bar, cost in zip(bars, climate_costs):
        ax4.text(cost + max(climate_costs)*0.02, bar.get_y() + bar.get_height()/2,
                f'${cost:,.0f}M', va='center', fontsize=10, fontweight='bold')
    
    ax4.invert_yaxis()
    ax4.set_xlabel('Estimated Annual Climate Cost ($M)', fontsize=11, fontweight='bold')
    ax4.set_title('2050 Financial Exposure\n(SSP5-8.5 Scenario)', 
                  fontsize=12, fontweight='bold', pad=10)
    ax4.xaxis.grid(True, alpha=0.3)
    
    # ========================================================================
    # PANEL 5 (Bottom-Right): COST AS % OF REVENUE
    # ========================================================================
    ax5 = fig.add_subplot(gs[2, 1])
    
    cost_pct = comp_sorted['cost_pct_revenue'].values
    
    bars = ax5.barh(companies, cost_pct, color='#9b59b6',
                    edgecolor='black', linewidth=1.5, alpha=0.85)
    
    # Highlight Starbucks
    bars[starbucks_idx].set_color('#3498db')
    bars[starbucks_idx].set_edgecolor('gold')
    bars[starbucks_idx].set_linewidth(3)
    
    # Add value labels
    for bar, pct in zip(bars, cost_pct):
        ax5.text(pct + 0.05, bar.get_y() + bar.get_height()/2,
                f'{pct:.2f}%', va='center', fontsize=10, fontweight='bold')
    
    ax5.invert_yaxis()
    ax5.set_xlabel('Climate Cost as % of Revenue', fontsize=11, fontweight='bold')
    ax5.set_title('Financial Materiality\n(Relative Impact)', 
                  fontsize=12, fontweight='bold', pad=10)
    ax5.xaxis.grid(True, alpha=0.3)
    
    # Add materiality threshold
    ax5.axvline(x=5.0, color='red', linestyle='--', linewidth=2, alpha=0.5)
    ax5.text(5.0, len(companies)*0.95, '5% Materiality\nThreshold', ha='right', 
             fontsize=8, bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))
    
    plt.tight_layout()
    plt.savefig(f"{output_dir}/04_Competitive_Benchmarking.png", dpi=300, bbox_inches='tight')
    plt.close()
    print("   ✓ Saved: 04_Competitive_Benchmarking.png")


4. Creating Competitive Benchmarking Chart...
   ✓ Saved: 04_Competitive_Benchmarking.png


In [64]:
# ============================================================================
# VIZ 5: EMISSION TRAJECTORY & NET-ZERO PATHWAY
# ============================================================================

print("\n5. Creating Emission Trajectory Visualization...")

# Check if trajectory_df exists from Step 17A
if 'trajectory_df' not in globals():
    print("   ⚠️ trajectory_df not found - run Step 17A first")
else:
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))
    fig.suptitle('STARBUCKS NET-ZERO EMISSION TRAJECTORY\nPathway to 2050 Climate Commitments', 
                 fontsize=16, fontweight='bold')
    
    # Get data
    years = trajectory_df['year'].values
    bau = trajectory_df['business_as_usual_MT'].values
    net_zero = trajectory_df['net_zero_MT'].values
    
    # ========================================================================
    # TOP CHART: EMISSION TRAJECTORY
    # ========================================================================
    
    # Plot trajectories
    ax1.plot(years, bau, marker='o', linewidth=3, markersize=10, 
             label='Business as Usual (+2%/year)', color='#e74c3c', linestyle='--', alpha=0.8)
    ax1.plot(years, net_zero, marker='s', linewidth=3, markersize=10,
             label='Net-Zero Pathway (Official Target)', color='#2ecc71', alpha=0.8)
    
    # Fill area between trajectories (emissions avoided)
    ax1.fill_between(years, bau, net_zero, alpha=0.2, color='green', 
                     label='Emissions Avoided')
    
    # Add milestone annotations
    ax1.annotate('2030 Target\n50% Reduction\n7.65 MT', 
                xy=(2030, net_zero[1]), xytext=(2028, net_zero[1] + 3),
                arrowprops=dict(arrowstyle='->', color='green', lw=2),
                fontsize=10, fontweight='bold',
                bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))
    
    ax1.annotate('2050 Net-Zero\n95% Reduction\n0.77 MT', 
                xy=(2050, net_zero[-1]), xytext=(2047, net_zero[-1] + 3),
                arrowprops=dict(arrowstyle='->', color='darkgreen', lw=2),
                fontsize=10, fontweight='bold',
                bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))
    
    # Add baseline
    ax1.axhline(y=15.3, color='gray', linestyle=':', linewidth=2, alpha=0.5)
    ax1.text(2024.5, 15.3 + 0.5, '2023 Baseline: 15.3 MT', 
             fontsize=9, style='italic', color='gray')
    
    # Formatting
    ax1.set_xlabel('Year', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Total Emissions (Million MT CO₂e)', fontsize=12, fontweight='bold')
    ax1.set_title('Emission Reduction Pathway', fontsize=13, fontweight='bold', pad=10)
    ax1.legend(fontsize=11, loc='upper right')
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, max(bau) * 1.1)
    
    # ========================================================================
    # BOTTOM CHART: EMISSIONS BY SCOPE (2024 vs 2030 vs 2050)
    # ========================================================================
    
    # Data for stacked bar chart
    milestone_years = [2024, 2030, 2050]
    scope_1 = [0.3, 0.15, 0.015]  # 50% reduction by 2030, 95% by 2050
    scope_2 = [1.2, 0.6, 0.06]
    scope_3 = [13.8, 6.9, 0.69]
    
    x_pos = np.arange(len(milestone_years))
    width = 0.5
    
    # Stacked bars
    p1 = ax2.bar(x_pos, scope_1, width, label='Scope 1 (Direct operations)', 
                 color='#e74c3c', edgecolor='black', linewidth=1.5)
    p2 = ax2.bar(x_pos, scope_2, width, bottom=scope_1,
                 label='Scope 2 (Purchased energy)', 
                 color='#f39c12', edgecolor='black', linewidth=1.5)
    p3 = ax2.bar(x_pos, scope_3, width, 
                 bottom=np.array(scope_1) + np.array(scope_2),
                 label='Scope 3 (Supply chain)', 
                 color='#3498db', edgecolor='black', linewidth=1.5)
    
    # Add total labels on top
    totals = [s1 + s2 + s3 for s1, s2, s3 in zip(scope_1, scope_2, scope_3)]
    for i, (x, total) in enumerate(zip(x_pos, totals)):
        ax2.text(x, total + 0.5, f'{total:.2f} MT', 
                ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    # Add reduction percentage labels
    reductions = ['Baseline', '-50%', '-95%']
    for i, (x, red) in enumerate(zip(x_pos, reductions)):
        ax2.text(x, -1.5, red, ha='center', va='top', fontsize=10, 
                fontweight='bold', color='green' if i > 0 else 'black')
    
    # Formatting
    ax2.set_ylabel('Emissions (Million MT CO₂e)', fontsize=12, fontweight='bold')
    ax2.set_title('Emission Reduction by Scope', fontsize=13, fontweight='bold', pad=10)
    ax2.set_xticks(x_pos)
    ax2.set_xticklabels(milestone_years, fontsize=11)
    ax2.legend(fontsize=10, loc='upper right')
    ax2.yaxis.grid(True, alpha=0.3)
    ax2.set_ylim(-2, max(totals) * 1.15)
    
    # Add key insight box
    insight = "Scope 3 (supply chain) represents\n90% of emissions - requires\nsupplier engagement & regenerative\nagriculture investment"
    ax2.text(0.02, 0.98, insight, transform=ax2.transAxes,
             bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8),
             fontsize=9, verticalalignment='top')
    
    plt.tight_layout()
    plt.savefig(f"{output_dir}/05_Emission_Trajectory.png", dpi=300, bbox_inches='tight')
    plt.close()
    print("   ✓ Saved: 05_Emission_Trajectory.png")


5. Creating Emission Trajectory Visualization...
   ✓ Saved: 05_Emission_Trajectory.png


In [65]:
import openpyxl
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows

print("\n" + "="*80)
print("CONSOLIDATING ALL RESULTS INTO MASTER EXCEL FILE")
print("="*80)

# Output file
output_file = f"{CONFIG['base_path']}/outputs/STARBUCKS_CLIMATE_RISK_ANALYSIS_MASTER.xlsx"

# Create Excel writer
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    
    # ========================================================================
    # SHEET 1: EXECUTIVE SUMMARY
    # ========================================================================
    
    summary_data = {
        'Metric': [
            'Analysis Period',
            'Scenarios Modeled',
            'Regions Analyzed',
            'Annual Coffee Volume (MT)',
            '',
            'PHYSICAL RISK (2050, Most Likely)',
            'Annual Profit at Risk',
            'Supply at Risk (%)',
            '',
            'TRANSITION RISK (2050, Most Likely)',
            'Annual Carbon Cost',
            'Carbon Price ($/tCO2)',
            '',
            'TOTAL CLIMATE COST (2050)',
            'Annual Exposure',
            '30-Year NPV',
            '% of Total Assets',
            '% of Total Revenue',
            '',
            'COMPETITIVE POSITION',
            'Vulnerability Score',
            'Rank vs Competitors',
            'Top-3 Concentration',
            'Diversification Score',
            '',
            'GEOGRAPHIC RESILIENCE',
            'Low-Risk Regions (%)',
            'Medium-Risk Regions (%)',
            'High-Risk Regions (%)',
        ],
        'Value': [
            '2024-2050',
            'SSP1-2.6, SSP2-4.5, SSP5-8.5',
            '29',
            f"{CONFIG['financial_params']['starbucks_annual_coffee_mt']:,.0f}",
            '',
            '',
            f"${combined_df[(combined_df['scenario_code']=='ssp245') & (combined_df['year']==2050)]['physical_risk_M'].values[0]:.1f}M",
            f"{combined_df[(combined_df['scenario_code']=='ssp245') & (combined_df['year']==2050)]['pct_supply_at_risk'].values[0]:.1f}%",
            '',
            '',
            f"${combined_df[(combined_df['scenario_code']=='ssp245') & (combined_df['year']==2050)]['carbon_cost_M'].values[0]:.1f}M",
            '$306/tCO2',
            '',
            '',
            f"${combined_df[(combined_df['scenario_code']=='ssp245') & (combined_df['year']==2050)]['total_climate_cost_M'].values[0]:.1f}M",
            f"${total_npv_df[total_npv_df['scenario']=='ssp245']['npv_30yr_M'].values[0]:.1f}M",
            f"{total_npv_df[total_npv_df['scenario']=='ssp245']['pct_of_total_assets'].values[0]:.2f}%",
            f"{total_npv_df[total_npv_df['scenario']=='ssp245']['pct_of_total_revenue'].values[0]:.2f}%",
            '',
            '',
            f"{competitor_df[competitor_df['company']=='Starbucks']['vulnerability_score'].values[0]:.2f}",
            '4 of 5 (Second-Best)',
            '17.5%',
            f"{competitor_df[competitor_df['company']=='Starbucks']['diversification_score'].values[0]:.2f}",
            '',
            '',
            '84.8%',
            '4.0%',
            '0.0%',
        ]
    }
    
    summary_df = pd.DataFrame(summary_data)
    summary_df.to_excel(writer, sheet_name='Executive Summary', index=False)
    
    # ========================================================================
    # SHEET 2: KEY METRICS BY SCENARIO
    # ========================================================================
    
    total_npv_df.to_excel(writer, sheet_name='NPV Summary', index=False)
    
    # ========================================================================
    # SHEET 3: SCENARIO FINANCIAL IMPACTS
    # ========================================================================
    
    scenario_summary.to_excel(writer, sheet_name='Scenario Summary', index=False)
    
    # ========================================================================
    # SHEET 4: PHYSICAL + TRANSITION COMBINED
    # ========================================================================
    
    combined_df.to_excel(writer, sheet_name='Combined Risks', index=False)
    
    # ========================================================================
    # SHEET 5: CARBON PRICING TRAJECTORY
    # ========================================================================
    
    carbon_costs_df.to_excel(writer, sheet_name='Carbon Costs', index=False)
    
    # ========================================================================
    # SHEET 6: REGIONAL DETAILED ANALYSIS
    # ========================================================================
    
    yield_df.to_excel(writer, sheet_name='Regional Yields', index=False)
    
    # ========================================================================
    # SHEET 7: REGIONAL RISK CONCENTRATION
    # ========================================================================
    
    regional_risk_sorted = yield_df[
        (yield_df['scenario'] == 'ssp585') & 
        (yield_df['period'] == 'mid_term')
    ].sort_values('profit_at_risk_M', ascending=False)
    
    regional_risk_sorted.to_excel(writer, sheet_name='Regional Concentration', index=False)
    
    # ========================================================================
    # SHEET 8: COMPETITOR BENCHMARKING
    # ========================================================================
    
    competitor_df.to_excel(writer, sheet_name='Competitor Analysis', index=False)
    
    # ========================================================================
    # SHEET 9: EMISSION TRAJECTORY
    # ========================================================================
    
    trajectory_df.to_excel(writer, sheet_name='Emissions Trajectory', index=False)

print(f"\n✓ Master Excel file created: {output_file}")
print("\nSheets included:")
print("  1. Executive Summary (Key metrics)")
print("  2. NPV Summary (Total climate NPV by scenario)")
print("  3. Scenario Summary (Physical + transition impacts)")
print("  4. Combined Risks (Physical + transition by year)")
print("  5. Carbon Costs (Carbon pricing trajectory)")
print("  6. Regional Yields (Detailed regional analysis)")
print("  7. Regional Concentration (Top vulnerable regions)")
print("  8. Competitor Analysis (Benchmarking vs peers)")
print("  9. Emissions Trajectory (Net-zero pathway)")

print("\n" + "="*80)


CONSOLIDATING ALL RESULTS INTO MASTER EXCEL FILE

✓ Master Excel file created: C:/Users/ibeha/OneDrive/Desktop/Climate/outputs/STARBUCKS_CLIMATE_RISK_ANALYSIS_MASTER.xlsx

Sheets included:
  1. Executive Summary (Key metrics)
  2. NPV Summary (Total climate NPV by scenario)
  3. Scenario Summary (Physical + transition impacts)
  4. Combined Risks (Physical + transition by year)
  5. Carbon Costs (Carbon pricing trajectory)
  6. Regional Yields (Detailed regional analysis)
  7. Regional Concentration (Top vulnerable regions)
  8. Competitor Analysis (Benchmarking vs peers)
  9. Emissions Trajectory (Net-zero pathway)



In [67]:
# ============================================================================
# SENSITIVITY ANALYSIS: KEY ASSUMPTIONS IMPACT
# ============================================================================

print("\n" + "="*80)
print("SENSITIVITY ANALYSIS: VALIDATING KEY ASSUMPTIONS")
print("="*80)

# ============================================================================
# 1. STRESS-TO-YIELD CONVERSION SENSITIVITY
# ============================================================================

print("\n" + "-"*80)
print("1. STRESS-TO-YIELD CONVERSION SENSITIVITY")
print("-"*80)

def convert_stress_to_yield_loss(stress_value, adjustment_factor=1.0):
    """
    Adjustable stress-to-yield conversion
    adjustment_factor: 0.8 = more conservative (lower losses)
                      1.0 = baseline
                      1.2 = more aggressive (higher losses)
    """
    if stress_value < 0.2:
        loss = stress_value * 0.5
    elif stress_value < 0.4:
        loss = 0.10 + (stress_value - 0.2) * 0.75
    elif stress_value < 0.6:
        loss = 0.25 + (stress_value - 0.4) * 1.25
    elif stress_value < 0.8:
        loss = 0.50 + (stress_value - 0.6) * 1.25
    else:
        loss = 0.75 + (stress_value - 0.8) * 1.0
    
    # Apply adjustment factor
    adjusted_loss = loss * adjustment_factor
    return min(adjusted_loss, 0.95)

# Test with key regions
test_regions = {
    'Colombia Huila (high-elev)': 0.027,
    'Brazil São Paulo (low-elev)': 0.156,
    'Sumatra Aceh (extreme)': 0.235
}

print("\nBASELINE YIELDS vs ADJUSTED SCENARIOS:")
print(f"{'Region':<35s} {'Base':<10s} {'-20% Loss':<10s} {'+20% Loss':<10s} {'Range':<15s}")
print("-"*70)

baseline_yields = {}
for region, stress in test_regions.items():
    base_yield = convert_stress_to_yield_loss(stress, 1.0) * 100
    conservative_yield = convert_stress_to_yield_loss(stress, 0.8) * 100
    aggressive_yield = convert_stress_to_yield_loss(stress, 1.2) * 100
    baseline_yields[region] = base_yield
    
    range_pct = aggressive_yield - conservative_yield
    print(f"{region:<35s} {base_yield:>8.1f}% {conservative_yield:>8.1f}% {aggressive_yield:>8.1f}% ±{range_pct/2:.1f}%")

# ============================================================================
# 2. FINANCIAL IMPACT SENSITIVITY
# ============================================================================

print("\n" + "-"*80)
print("2. FINANCIAL IMPACT SENSITIVITY (Most Likely Scenario - 2050)")
print("-"*80)

# Base case from your analysis
base_physical_risk = 158.8
base_carbon_cost = 2295.0
base_total = base_physical_risk + base_carbon_cost
base_npv = 27624.7

discount_rate = 0.08

print(f"\nBASE CASE (Delayed Transition, 2050):")
print(f"  Physical risk: ${base_physical_risk:.1f}M")
print(f"  Carbon cost: ${base_carbon_cost:.1f}M")
print(f"  Total annual: ${base_total:.1f}M")
print(f"  30-year NPV (8%): ${base_npv:.1f}M")

print(f"\n{'Parameter':<40s} {'Scenario':<20s} {'Annual Impact':<15s} {'NPV 30yr':<15s} {'vs Base':<15s}")
print("-"*85)

# Scenario 1: Physical risk varies by ±30%
for adj, label in [(0.7, '-30% (High adaptation)'), (1.0, 'Baseline'), (1.3, '+30% (Low adaptation)')]:
    adj_phys = base_physical_risk * adj
    adj_total = adj_phys + base_carbon_cost
    adj_npv = sum([adj_total / ((1 + discount_rate) ** t) for t in range(1, 31)])
    diff = adj_npv - base_npv
    diff_pct = (diff / base_npv) * 100
    print(f"{'Physical Risk Adjustment':<40s} {label:<20s} ${adj_total:>12.1f}M ${adj_npv:>12.1f}M {diff_pct:>13.1f}%")

print()

# Scenario 2: Carbon price varies by ±25%
for adj, label in [(0.75, '-25% (Slower transition)'), (1.0, 'Baseline'), (1.25, '+25% (Faster transition)')]:
    adj_carbon = base_carbon_cost * adj
    adj_total = base_physical_risk + adj_carbon
    adj_npv = sum([adj_total / ((1 + discount_rate) ** t) for t in range(1, 31)])
    diff = adj_npv - base_npv
    diff_pct = (diff / base_npv) * 100
    print(f"{'Carbon Cost Adjustment':<40s} {label:<20s} ${adj_total:>12.1f}M ${adj_npv:>12.1f}M {diff_pct:>13.1f}%")

print()

# Scenario 3: Discount rate varies
for rate, label in [(0.06, '6% (Lower opportunity cost)'), (0.08, '8% (Baseline)'), (0.10, '10% (Higher opportunity cost)')]:
    adj_npv = sum([base_total / ((1 + rate) ** t) for t in range(1, 31)])
    diff = adj_npv - base_npv
    diff_pct = (diff / base_npv) * 100
    print(f"{'Discount Rate':<40s} {label:<20s} ${base_total:>12.1f}M ${adj_npv:>12.1f}M {diff_pct:>13.1f}%")

# ============================================================================
# 3. COMPETITIVE ADVANTAGE SENSITIVITY
# ============================================================================

print("\n" + "-"*80)
print("3. COMPETITIVE ADVANTAGE HOLDS ACROSS SCENARIOS")
print("-"*80)

starbucks_profit_risk = 32  # From earlier analysis (top 5 regions)
competitors_profit_risk = 65  # From earlier analysis

print(f"\nCOMPETITIVE ADVANTAGE: Starbucks vs Low-Elevation Competitors")
print(f"  Baseline Starbucks profit at risk: ${starbucks_profit_risk:.0f}M")
print(f"  Baseline Competitors profit at risk: ${competitors_profit_risk:.0f}M")
print(f"  Advantage: ${competitors_profit_risk - starbucks_profit_risk:.0f}M\n")

print(f"{'Scenario':<40s} {'Starbucks':<15s} {'Competitors':<15s} {'Advantage':<15s}")
print("-"*85)

scenarios = [
    (0.7, '-30% Stress (High adaptation)'),
    (1.0, 'Baseline'),
    (1.3, '+30% Stress (Low adaptation)')
]

for adj, label in scenarios:
    sbux_adj = starbucks_profit_risk * adj
    comp_adj = competitors_profit_risk * adj
    advantage = comp_adj - sbux_adj
    print(f"{label:<40s} ${sbux_adj:>13.1f}M ${comp_adj:>13.1f}M ${advantage:>13.1f}M")

# ============================================================================
# ============================================================================
# 4. PRICE ELASTICITY SENSITIVITY (CORRECTED)
# ============================================================================

print("\n" + "-"*80)
print("4. PRICE ELASTICITY SENSITIVITY")
print("-"*80)

# Use actual supply shock from yield impact
base_volume_at_risk = 7701  # MT (from your 2050 data)
total_volume = CONFIG['financial_params']['starbucks_annual_coffee_mt']
supply_shock_pct = (base_volume_at_risk / total_volume)  # This is the % supply lost

current_revenue_per_kg = (CONFIG['financial_params']['coffee_revenue'] / 
                          (CONFIG['financial_params']['starbucks_annual_coffee_mt'] * 1000))

print(f"\nSupply shock: {supply_shock_pct*100:.2f}% of total volume ({base_volume_at_risk:,} MT)")
print(f"Revenue per kg: ${current_revenue_per_kg:.2f}\n")

print(f"{'Elasticity':<25s} {'Label':<25s} {'Price Increase':<18s} {'Demand Loss':<15s}")
print("-"*83)

elasticities = [(-0.5, 'Very Inelastic'), (-0.8, 'Baseline (Literature)'), (-1.2, 'Elastic')]

for elast, label in elasticities:
    # Price elasticity formula: %ΔP = -%ΔQ / elasticity
    price_increase_pct = (-supply_shock_pct * 100) / elast
    # Demand loss: %ΔQ = elasticity × %ΔP
    demand_loss_pct = abs(elast * (price_increase_pct / 100)) * 100
    
    print(f"{label:<25s} {label:<25s} {price_increase_pct:>16.2f}% {demand_loss_pct:>13.2f}%")

print("\nInterpretation:")
print("  • Very Inelastic (-0.5): Coffee is essential → small demand loss (3%)")
print("  • Baseline (-0.8): 4.5% supply reduction requires 5.6% price increase")
print("  • Elastic (-1.2): Consumers very price-sensitive → 3.8% demand loss")

# ============================================================================
# 5. SUMMARY: MATERIALITY ACROSS ALL SCENARIOS
# ============================================================================

print("\n" + "="*80)
print("MATERIALITY ASSESSMENT ACROSS SENSITIVITY SCENARIOS")
print("="*80)

print(f"\nTotal Assets: ${CONFIG['financial_params']['total_assets']/1e9:.1f}B")
print(f"Materiality Threshold (5%): ${CONFIG['financial_params']['total_assets'] * 0.05 / 1e9:.1f}B\n")

print(f"{'Scenario':<40s} {'30-Yr NPV':<15s} {'% of Assets':<15s} {'Material?':<12s}")
print("-"*82)

scenarios_materiality = [
    (base_npv * 0.7, 'Conservative (-30% scenario)'),
    (base_npv * 1.0, 'Baseline'),
    (base_npv * 1.3, 'Aggressive (+30% scenario)')
]

for npv, label in scenarios_materiality:
    pct_assets = (npv * 1e6 / CONFIG['financial_params']['total_assets']) * 100
    material = "YES ✓" if pct_assets > 5 else "NO"
    print(f"{label:<40s} ${npv:>12.1f}M {pct_assets:>13.2f}% {material:<12s}")

print("\n" + "="*80)
print("KEY INSIGHT: Climate risk remains MATERIAL across all reasonable scenarios")
print("="*80)

# ============================================================================
# 6. EXPORT SENSITIVITY RESULTS
# ============================================================================

sensitivity_summary = pd.DataFrame({
    'Assumption': [
        'Physical Risk -30% (High Adaptation)',
        'Physical Risk Baseline',
        'Physical Risk +30% (Low Adaptation)',
        'Carbon Cost -25% (Slower Transition)',
        'Carbon Cost Baseline',
        'Carbon Cost +25% (Faster Transition)',
        'Discount Rate 6%',
        'Discount Rate 8% (Baseline)',
        'Discount Rate 10%'
    ],
    'Annual_Impact_M': [
        base_physical_risk*0.7 + base_carbon_cost,
        base_total,
        base_physical_risk*1.3 + base_carbon_cost,
        base_physical_risk + base_carbon_cost*0.75,
        base_total,
        base_physical_risk + base_carbon_cost*1.25,
        base_total,
        base_total,
        base_total
    ],
    'NPV_30yr_M': [
        sum([((base_physical_risk*0.7 + base_carbon_cost) / ((1 + 0.08) ** t)) for t in range(1, 31)]),
        base_npv,
        sum([((base_physical_risk*1.3 + base_carbon_cost) / ((1 + 0.08) ** t)) for t in range(1, 31)]),
        sum([((base_physical_risk + base_carbon_cost*0.75) / ((1 + 0.08) ** t)) for t in range(1, 31)]),
        base_npv,
        sum([((base_physical_risk + base_carbon_cost*1.25) / ((1 + 0.08) ** t)) for t in range(1, 31)]),
        sum([(base_total / ((1 + 0.06) ** t)) for t in range(1, 31)]),
        base_npv,
        sum([(base_total / ((1 + 0.10) ** t)) for t in range(1, 31)])
    ]
})

sensitivity_summary.to_csv(f"{CONFIG['base_path']}/outputs/sensitivity_analysis.csv", index=False)

print("\n✓ Saved sensitivity analysis to outputs/sensitivity_analysis.csv")


SENSITIVITY ANALYSIS: VALIDATING KEY ASSUMPTIONS

--------------------------------------------------------------------------------
1. STRESS-TO-YIELD CONVERSION SENSITIVITY
--------------------------------------------------------------------------------

BASELINE YIELDS vs ADJUSTED SCENARIOS:
Region                              Base       -20% Loss  +20% Loss  Range          
----------------------------------------------------------------------
Colombia Huila (high-elev)               1.4%      1.1%      1.6% ±0.3%
Brazil São Paulo (low-elev)              7.8%      6.2%      9.4% ±1.6%
Sumatra Aceh (extreme)                  12.6%     10.1%     15.1% ±2.5%

--------------------------------------------------------------------------------
2. FINANCIAL IMPACT SENSITIVITY (Most Likely Scenario - 2050)
--------------------------------------------------------------------------------

BASE CASE (Delayed Transition, 2050):
  Physical risk: $158.8M
  Carbon cost: $2295.0M
  Total annual: $245